# Environment Variable

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
env

{'COLORTERM': 'truecolor',
 'COMMAND_MODE': 'unix2003',
 'HOME': '/Users/picetrp',
 'LANG': 'en_GB.UTF-8',
 'LOGNAME': 'picetrp',
 'PATH': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/.venv/bin:/Users/picetrp/.langflow/uv:/Users/picetrp/Downloads/google-cloud-sdk/bin:/Users/picetrp/.local/bin:/Users/picetrp/.pyenv/shims:/Users/picetrp/.pyenv/bin:/opt/homebrew/opt/jpeg/bin:/Users/picetrp/opt/anaconda3/condabin:/Users/picetrp/.nvm/versions/node/v16.20.2/bin:/Users/picetrp/.rubies/ruby-3.2.2/bin:/opt/homebrew/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Library/TeX/texbin:/Users/picetrp/.cargo/bin:/Users/picetrp/Library/Application Support/JetBrains/Toolbox/scripts:/Users/picetrp/development/flut

In [3]:
import os
print(os.getenv("PWD"))
print(os.getcwd())

/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag
/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/notebooks


In [4]:
from pydantic import Field, validator
from pydantic_settings import BaseSettings, SettingsConfigDict
import os

class Settings(BaseSettings):
    # * langsmith setting
    langsmith_tracing: str
    langsmith_endpoint: str
    langsmith_api_key: str
    langsmith_project: str

    # * openai settings
    openai_api_key: str
    google_api_key: str
    embedding_model: str = "all-MiniLM-L6-v2"
    
    # * qdrant settings
    qdrant_cloud_api_key: str
    qdrant_cloud_url: str = "https://cloud.qdrant.io"
    qdrant_collection_name: str = "demo_collection"

    # * huggingface token
    huggingface_token: str
    
    # * chunking settings
    chunk_size: int = 1000
    chunk_overlap: int = 200
    top_k_results: int = 5
    
    # * api settings
    api_host: str = "0.0.0.0"
    api_port: int = 8000
    
    # * file settings
    root_dir: str = "/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag"
    upload_dir: str = os.path.join(root_dir, "data/uploads")

    model_config = SettingsConfigDict(
        env_file= os.path.join(root_dir, ".env"),
        env_file_encoding="utf-8",
    )
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        os.makedirs(self.upload_dir, exist_ok=True)

settings = Settings()

# Document

In [120]:
root_path = settings.root_dir
data_dirpath = settings.upload_dir
data_dirpath

'/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads'

In [121]:
from typing import List, Set

def get_filtered_files_by_extension(folder_path: str,
                                    extensions: Set[str] = {".pdf", ".txt", ".docx", ".doc"}
                                   ) -> List[str]:
    extensions = {ext.lower() for ext in extensions}  # Ensure all extensions are lowercase

    try:
        all_files = os.listdir(folder_path)
    except FileNotFoundError:
        raise ValueError(f"Folder not found: {folder_path}")

    return [
        f for f in all_files
        if os.path.isfile(os.path.join(folder_path, f)) and os.path.splitext(f)[1].lower() in extensions
    ]

doc_files = get_filtered_files_by_extension(data_dirpath)
doc_files = [os.path.join(data_dirpath, filename) for filename in doc_files]
doc_files

['/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/Template-short-Paper-3-1.pdf',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/แบบฟอร์ม_บ1_IRB.pdf',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/ข้อมูลที่ต้องเขียนในส่วนของวิธีดำเนินการวิจัยโดยสังเขป.pdf',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/sit_kmutt_แนะนำการทำ_Short_Paper.txt',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายชื่อคณะกรรมการสอบการศึกษาค้นคว้าอิสระขั้นสุดท้าย.pdf',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/การโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่อง-IT.docx',
 '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/การโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่อง-BIS.docx']

### split docs

In [122]:
import os
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from IPython.display import display, Markdown
from tqdm.notebook import tqdm

def preprocess_document(file_path):
    """
    Load a document based on its file extension and return LangChain Document(s).
    
    Supported:
    - .txt
    - .pdf
    - .docx
    """
    ext = os.path.splitext(file_path)[1].lower()
    
    if ext == ".txt":
        loader = TextLoader(file_path, encoding="utf-8")
    elif ext == ".pdf":
        loader = PyPDFLoader(file_path)
    elif ext == ".doc" or ".docx":
        loader = UnstructuredWordDocumentLoader(file_path, mode="paged")
    else:
        raise ValueError(f"Unsupported file type: {ext}")
    
    return loader.load()

In [123]:
all_docs = []
for doc_path in tqdm(doc_files):
    doc_content = preprocess_document(doc_path)
    all_docs.extend(doc_content)

  0%|          | 0/8 [00:00<?, ?it/s]

Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.uns

In [124]:
len(all_docs)

16

### Observe data in each type

#### pdf

In [33]:
example_doc = all_docs[0][0]
doc_metadata = example_doc.metadata
doc_content = example_doc.page_content

In [34]:
display(doc_metadata)
display(Markdown(doc_content))

{'producer': 'Microsoft® Word for Microsoft 365',
 'creator': 'Microsoft® Word for Microsoft 365',
 'creationdate': '2025-05-18T12:50:12+07:00',
 'author': 'WALAIPUN PORNWIROON',
 'moddate': '2025-05-18T12:50:12+07:00',
 'source': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf',
 'total_pages': 2,
 'page': 0,
 'page_label': '1'}

ขั้นตอนที่ต้องปฏิบัติก่อนถึงวันสอบ Short Paper ขั้นสุดท้าย (เฉพาะนักศึกษารหัสนำหน้า 63XXXXXXXXX 
ขึ้นไป) 
            1.นักศึกษาต้องแก้ไขตามคำแนะนำของอาจารย์ที่ปรึกษาและกรรมการภายในคณะฯ ให้เสร็จสิ้นเรียบร้อย      
ทั้งนี้ระบบจะดึงบทความ Short Paper ฉบับล่าสุดให้กับผู้ทรงคุณวุฒิภายนอกอัตโนมัติ 
2. ดำเนินการ Upload คลิป VDO บรรยายผลการศึกษา Short Paper ความยาวประมาณ ไม่เกิน 5 นาที      
ไฟล์ PowerPoint และบทความ Short Paper ฉบับสมบูรณ์ที่ผ่านจากกรรมการภายในคณะ 2 ท่านเรียบร้อยแล้ว ให้ 
Upload link ใน Google Drive เท่านั้น ไม่อนุญาต Upload link ใน YouTube โดยนำ link ที่จัดเก็บข้อมูลดังกล่าว 
Upload ให้กรรมการทั้ง 3 ท่าน ได้ที่ http://www2.sit.kmutt.ac.th/isreport/New/  โดยกรรมการภายนอกเริ่มอ่าน 
(เนื่องจากกรรมการภายนอก 1 ท่านต้องอ่านบทความ Short Paper หลายชื่อเรื่อง ดังนั้นขอให้นักศึกษาดำเนินการให้แล้ว
เสร็จภายในระยะเวลาที่ประกาศไว้ในกำหนดการ)  
 3. กรุณาตรวจสอบเวลาสอบ https://www.sit.kmutt.ac.th/short-paper/  กรณีที่นักศึกษาไม่เข้าสอบในวันเสาร์
ที่ 24 พฤษภาคม 2568 ตามช่วงเวลาที่ประกาศไว้  ผลการประเมิน Short paper “ไม่ผ่าน” ทันที 
 
 
 
ข้อปฏิบัติการสอบ Short Paper ขั้นสุดท้าย  (วันเสาร์ที่ 24 พฤษภาคม 2568) 
1. สอบผ่านระบบ Zoom โดยเข้าที่ URL: https://kmutt-ac-th.zoom.us/j/96941702766 
2. Rename ชื่อดังนี้ ห้องสอบ_รหัสนักศึกษาสองตัวหน้า_รหัสนักศึกษาสามตัวท้าย_ชื่อภาษาไทย  
เช่น A_55_999_คุณิตา เป็นต้น   
            3. เข้าระบบ Zoom ก่อนถึงเวลาสอบอย่างน้อย 15 นาที โดยรออยู่ใน Waiting Room และเจ้าหน้าที่จะเชิญ 
นักศึกษาเข้าห้องสอบ (หมายเหตุ การเข้าสอบของแต่ละบุคคลอาจใช้เวลาสอบมากหรือน้อยกว่าที่กำหนดตารางประกาศ 
นักศึกษาโปรดรอการเรียกเข้าสอบ)  
4. เปิดกล้องตลอดเวลาการสอบ (โปรดแต่งกายสุภาพ)   
5. สอบปากเปล่า  
6. เสร็จสิ้นการสอบและแก้ไขตามคำแนะนำของคณะกรรมการสอบ (ตามกำหนดการ Short Paper) 
  
 
กำหนดการสอบ Present Short Paper ขั้นสุดท้าย  (วันเสาร์ที่ 24 พฤษภาคม 2568) 
1. การสอบแบ่งห้องสอบออกเป็น 3 ห้อง คือ ห้อง A ห้อง B และห้อง C                           
2. ให้นักศึกษาเข้าสอบตามวันและเวลาที่กำหนด     
• ห้อง A        
เริ่มสอบช่วงเช้า 09.00-12.00 น.     เริ่มสอบช่วงบ่าย 13.00-15.00 น. 
• ห้อง B  
เริ่มสอบช่วงเช้า 09.00-12.00 น.     เริ่มสอบช่วงบ่าย 13.00-15.00 น. 
• ห้อง C  
เริ่มสอบช่วงเช้า 09.00-12.00 น.     เริ่มสอบช่วงบ่าย 13.00-15.00 น. 
 
3. ให้นักศึกษาปรับแก้ชื่ออาจารย์ที่ปรึกษาในบทคัดย่อภาษาไทย และภาษาอังกฤษ เดิมจากอาจารย์ 
ที่ปรึกษาประจำวิชาเป็นชื่ออาจารย์ที่ปรึกษาตามไฟล์ excel ที่ประกาศรายชื่อคณะกรรมการสอบค้นคว้าอิสระ               
ขั้นสุดท้าย Short Paper   
4. กรณีชื่อเรื่องใน Short Paper มีการเปลี่ยนแปลงทั้งภาษาไทย และ (หรือ) ภาษาอังกฤษ หลังจากให้

#### txt

In [20]:
example_doc = all_docs[4]
# display(example_doc)
print()

for doc in example_doc:
    doc_metadata = doc.metadata
    doc_content = doc.page_content
    display(doc_metadata)
    display(Markdown(doc_content))

{'source': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/sit_kmutt_แนะนำการทำ_Short_Paper.txt'}

video_url: https://www.youtube.com/watch?v=i2xJ2FjXNZ4
video_name: "SIT KMUTT | แนะนำการทำ Short Paper"
video_channel_url: "https://www.youtube.com/@SIT.Family"

สวัสดีค่ะสำหรับนักศึกษาท่านใดที่สงสัย ว่าชอตเปอร์นั้นคืออะไรแล้วต้องทำยังไง และมีกระบวนการทำยังไงนั้นวันนี้เรามีคำ ตอบในคลิปนี้ค่ะอันดับแรก Short  Paper  คืออะไรจริงๆแล้วเนี่ยก็คือเป็นบทความ วิชาการนะคะที่มีเนื้อหาครอบคลุมทั้งหมด ของการศึกษาเฉพาะเรื่องของนักศึกษาแต่ละ ท่านนะคะวัตถุประสงค์ของการทำชต Paper  นั้นก็คือการเผยแพร่เนื้อหาการศึกษาเฉพาะ เรื่องที่เราทำนั่นเองนะคะซึ่งเป็นการค้น คว้าอิสระเชิงประยุกต์การใช้ความรู้ใน วิชาชีพที่เราเรียนมาในหลักสูตรทั้งหมด นั่นเองค่ะตัวอย่างเช่นการปรับปรุงกระบวน การทำงานใดๆก็ตามที่ให้มีการทำงานที่ดี ขึ้นการนำเทคโนโลยีมาใช้แก้ไขปัญหาในการ ทำงานของเราที่ทำงานในปัจจุบันนะคะหรือ การคิดค้นพัฒนาเครื่องมือหรือทฤษฎีใหม่ๆ ขึ้นมาเพื่อสามารถตอบโจทย์ความต้องการณ ปัจจุบันได้อันนี้ก็สามารถนำมาทำเป็นโครง การศึกษาเฉพาะเรื่องได้นะคะซึ่งชต Paper  นั้นนะคะก็จะเป็นบทความที่บรรยายเกี่ยว กับการศึกษาเฉพาะเรื่องที่เราทำนะคะซึ่ง จะมีความยาวไม่เกิน 10 หน้าโดยที่ไม่รวม บทขัดย่อนะคะสำหรับกระบวนการทำชต Paper  นะคะหลายคนสงสัยว่าจะมีกระบวนการสเต็ปการ ทำงานของมันอย่างไรบ้างนะคะเดี๋ยวเราจะมี อ่อตัว QR  Code ขึ้นให้บนหน้าจอนะคะเรา จะสามารถเข้าไปเห็นกระบวนการการทำชอตเปอร ได้นะคะจะมีตารางการส่งงานต่างๆให้เรา เห็นนะคะแล้วก็สามารถที่จะอ่าทำงานนะคะทำ ตามสเต็ปที่เค้าวางไว้ให้ได้เลยนะคะซึ่ง ในการเขียน Short  Paper นะคะจะใช้เตของ คณะที่จัดไว้ให้เท่านั้นนะคะก็สามารถ ดาวน์โหลดได้กันที่เว็บไซต์ของคณะและวิธี การส่งชต Paper นะคะเราก็จะมีเอ่อ เว็บไซต์นะคะในการส่งชต Paper นะคะซึ่งจะ มีการส่งเา้าเรียกว่า is  Report นะคะ หรือว่า independent  study  Report นั่น เองนะคะส่งเข้าไปลิงก์ตามที่ขึ้นให้ด้าน ล่างนี้นะคะสำหรับนักศึกษาที่เข้ามาตั้ง แต่เอ่อเดือนมกราคมปี  2565 เป็นต้นไปนะคะข้อควรระวังอย่างนึง การทำชอตเปอร์หรือการศึกษาโครงการเฉพาะ เรื่องของนักศึกษาที่เข้ามาตั้งแต่มกราคม  2565 ย้ำนะคะจะต้องผ่านการอบรมจริยธรรม การวิจัยทั้งทฤษฎีและปฏิบัติก่อนการทำงาน นะคะโดยการอบรมนั้นนะคะจะแบ่งเป็นเป็น 2  ส่วนหลักนะคะส่วนแรกก็คือการอบรมนะคะ  Research  integrity อันนี้ทุกคนขีดเส้น ใต้ทุกคนจะต้องอบรมผ่านทั้งทฤษฎีและ ปฏิบัตินะคะและเราก็จะมีคลิปเ่อสั้นๆอีก อันนึงนะคะที่จะมาชี้แจงว่าการอบรมนะคะ หรือว่าการอบรมจริยธรรมการวิจัยเนี่ยจะ ต้องอบรมได้ที่ไหนบ้างแล้วก็ส่งเ่อผลการ อบรมได้ที่ไหนบ้างเพื่อเป็นการยืนยันว่า เรามีความพร้อมในการทำโครงการเฉพาะเฉพาะ เรื่องนะคะส่วนอีกอันนึงนั้นนะคะที่ขาด ไม่ได้สำหรับนักศึกษาท่านใดที่ทำโครงการ เฉพาะเรื่องที่เกี่ยวกับคนนะคะไม่ว่าจะ ใช้ข้อมูลเกี่ยวกับคนมีการเ่าทำเซอร์เวย์ แบบสอบถามสัมภาษณ์ที่ต้องใช้มนุษย์นะคะใน การทำงานของเรานั้นนะคะจะต้องผ่านอบรม irb  นะคะ irb นั้นนะคะเป็นการอบรมจริตธรรมการ วิจัยในมนุษย์นั่นเองนะคะแต่อย่างน้อย ระยะเวลาในการอบรม irb นะคะจะต้องเสร็จ ก่อนการทำการเก็บข้อมูลจากมนุษย์นั่นเอง ค่ะสำหรับเรื่องถัดไปนะคะกระบวนการทำชต  Paper นักศึกษาหลายคนก็สงสัยเหมือนกัน ว่ามันมีสเต็ปการทำงานยังไงนะคะในการทำชอ  Paper นะคะสเต็ปแรกเลยนะคะที่ทุกคนจะ ต้องทำคือหาที่ปรึกษาในการทำชต Paper นะ คะในเรื่องที่เราสนใจนะคะต้องใช้ผู้ เชี่ยวชาญด้านไหนเราก็ต้องหา advisor  หรือที่ปรึกษาที่มีความเชี่ยวชาญด้านนั้น นะคะและไม่จำเป็นที่จะต้องเป็นอาจารย์ที่ สอน workshop เท่านั้นด้วยนะคะเราเปิด กว้างให้นักศึกษาสามารถที่จะหาเ่าที่ ปรึกษาได้นะคะจากหลากหลายวิชาเลยนะคะไม่ จำเป็นจะต้องจะต้องเป็นวิชาเฉพาะ workshop  เท่านั้นนะคะโดยมีข้อแม้อย่างนึงนะคะคือ คุณสมบัติของอาจารย์ที่ปรึกษานั้นนะคะจะ ต้องมีวุฒิที่เป็นดอกเตอร์ขึ้นไปนะคะหรือ ว่าเ่าวุฒิในการจบการศึกษาปริญญาเอกนั่น เองนะคะแล้วมีผลงานทางวิชาการ 3 ชิ้นขึ้น ไปใน 5 ปีย้อนหลังพอเราได้ที่ปรึกษาแล้ว เราก็นัดคุยกันนะคะว่าหัวข้อที่เราจะทำ โครงการเฉพาะเรื่องนั้นเนี่ยเราจะทำอะไร นะคะโดยก็ปรึกษากันกับที่ปรึกษาแล้วถ้าลง ตัวแล้วได้หัวข้อแล้วนะคะเราก็จะจัดทำใบ บอ 1 นะคะใบบอ 1 ก็เหมือนใบชี้แจงว่าเรา จะทำเอ่อโครงการศึกษาเฉพาะเรื่องหัวข้อ อะไรนะคะโดยจำเป็นจะต้องมีลายเซ็นที่ ปรึกษากำกับนะคะเพื่อยืนยันว่าที่ปรึกษา นั้นรับรองหัวข้อของเราแล้วนะคะจากนั้นก็ จะส่งใบบ 1 ขึ้นระบบของ is  Report นั่น เองนะคะพอเราได้อ่าหัวข้อแล้วก็จะดำเนิน การทำอ่าโครงการเฉพาะเรื่องของเรานะคะ ดำเนินการจัดทำไปเรื่อยๆนะคะในช่วงแรกเรา ก็จะให้ส่งร่างบทความนะคะให้กับอาจารย์ ที่ปรึกษาประเมินครั้งที่ 1 นะคะตามแต่ ช่วงเวลาที่อาจารย์ที่ปรึกษากำหนดหลังจาก นั้นนะคะก็แก้ไขบทความตามคอมเมนต์หรือว่า ตามคำแนะนำของที่ปรึกษาให้เรียบร้อยนะคะ พอแก้เรียบร้อยก็ส่งให้อาจารย์ที่ปรึกษา ดูเป็นรอรอบที่ 2 นะคะก็คือเป็นการ ประเมินครั้งที่ 2 นั่นเองนะคะตามช่วง เวลาที่อาจารย์ปรึกษาอ่าที่ปรึกษากำหนด ให้เรานะคะพอทุกอย่างเสร็จละจากอ่าทางเรา และทางที่ปรึกษาทั้ง 2 คนเ่อ approve นะ คะหรือว่าคุณภาพของงานเนี่ยเป็นที่พึงพอ ใจของทั้ง 2 คนแล้วทั้งตัวนักศึกษาเองและ ทั้งตัวอาจารย์แล้วนะคะขั้นตอนถัดไปนะคะ ก็จะส่งให้คณะกรรมการประเมินทั้ง 3 ท่าน นะคะซึ่งคณะกรรมการเนี่ยนะคะก็จะประกอบไป ด้วยอาจารย์เอ่อจากคณะเรา 1 ท่านนะคะแล้ว ก็จะมีผู้ทรงคุณวุฒิภายนอกอีก 1 ท่านนะคะ ที่จะทำการอ่านบทความของเราและประเมินบท ความของเรานะคะดังนั้นในขั้นตอนนี้จะต้อง ระวังนะคะบทความที่ถูกส่งในการประเมิน ครั้งที่ 3 นี้นะคะจะต้องเป็นบทความที่ สมบูรณ์แล้วเท่านั้นนะคะและมีการจัด  Format อย่างดีเรียบร้อยแล้วเท่านั้นถึง จะสามารถส่งเข้าประเมินให้คณะกรรมการ 3  ท่านได้อ่านกันได้นะคะหลังจากนั้นเนี่ยก คณะกรรมการทั้ง 3 ท่านก็จะทำการทยอยอ่าน บทความของเรานะคะประเมินบทความของเรา พร้อมกับมีข้อเสนอแนะคะว่าให้เราแก้ไขตรง ไหนอะไรบ้างนะคะจากนั้นเราก็ทำการแก้ไขบท ความตามคำแนะนำของกรรมการทั้ง 3 ท่านแล้ว ก็ส่งเข้าระบบเป็นครั้งสุดท้ายนะคะเมื่อ บทความผ่านการประเมินแล้วนะคะก็คือพอทั้ง  3 ท่านเห็นข้อแก้ไขของเราแล้วเรียบร้อย ทั้ง 3 ท่านก็จะมาอ่าดูว่าผ่านเกณฑ์การ ประเมินหรือยังนะคะกรรมการทั้ง 3 ท่านก็ จะต้องให้ผลการประเมินเป็นผ่านพอผลการ ประเมินผ่านทั้ง 3 ท่านแล้วนะคะเราก็มี หน้าที่ที่จะเริ่มจัดทำข้อมูลเพื่อจะนำมา เสนอกรรมการทั้ง 3 ท่านในช่วงการสอบขั้น สุดท้ายนะคะโดยเอ่อข้อมูลที่จะต้องจัด เตรียมนะคะก็จะมีไฟล์นำเสนอข้อมูลนะคะก็ จะเป็น PowerPoint ก็ได้หรือสไลด์แบบไหน ก็ได้นะคะหรือว่าจะเป็น PDF ไล์ก็ได้นะคะ แนบ URL ผ่านระบบเข้ามานะคะพร้อมกับ วิดีโอนำเสนอที่มีความยาว 5 นาทีก็คืออัด วดีโอนำเสนอ Power  Power ของเรานั่นแหละ นะคะ 5 นาทีให้เรียบร้อยจากนั้นก็ส่ง ลิงก์เข้ามานะคะในระบบของ is  Report นะ คะพอทุกอย่างส่งเสร็จสิ้นเรียบร้อยแล้วก็ จะถึงขั้นตอนสุดท้ายคือขั้นตอนการสอบขั้น สุดท้ายนั่นเองนะคะอันนี้จะเป็นขั้นตอน การสอบปากเปล่านะคะก็คือนักศึกษาทุกคนจะ ต้องเข้ามาสอบปากเปล่ากับกรรมการทั้ง 3  ท่านนะคะก็เ่อเตรียมการอ่านำเสนองานของ เราได้เลยนะคะตามที่คณะกรรมการกำหนดนะคะ พอผ่านอ่าการสอบขั้นสุดท้ายแล้วถ้ามีสิ่ง ให้แก้ไขอีกนะคะก็ให้เราทำการแก้ไขแล้วก็ ส่งบทความกลับเข้ามาในระบบใหม่แต่ถ้าเกิด ว่าไม่มีอะไรที่ต้องแก้ไขก็คือนั่นหมาย ถึงว่าชอตเปอรของเรานั้นผ่านแล้วนะคะอัน นี้ก็เป็นกระบวนการคร่าวๆของการทำช Paper  ค่ะสำหรับข้อสงสัยถัดไปที่เป็นคำถามยอด นิยมสำหรับนักศึกษาเลยนะคะก็คือส่วน ประกอบของชอต Paper จะมีอะไรบ้างจะต้อง เขียนอะไรเข้าไปในชอต Paper เนื่องจากว่า ชอตเปเปอร์เนี่ยเป็นบทความเชิงวิชาการที่ จะใช้อธิบายโครงการเฉพาะเรื่องที่เราทำ อยู่นะคะดังนั้นรายละเอียดส่วนใหญ่ก็จะ เป็นรายละเอียดเกี่ยวกับงานที่เราทำนั่น เองนะคะโดยทั่วๆไปแล้วนะคะถ้าเกิดว่าใคร เป็นมือใหม่ในการเขียนก็สามารถทำตามคำแนะ นำนี้ได้เลยนะคะก็คือส่วนประกอบก็จะมี ประกอบไปด้วย 1  +  5 หัวข้อหลักนั่นเอง อ่างงล่ะสิว่าทำไมถึงมีติ่ง 1  +  5 ใช่ มั้ยคะ 1 ที่บวกเข้ามานั้นก็คือบทคัดย่อ ซึ่งบทคัดย่อเนี่ยจะจะไม่รวมอยู่ในเนื้อ หานะคะก็คือเนื้อหาเนี่ยเราอนุญาตให้มี ประมาณ 10 หน้านะคะซึ่งไม่รวมบทคัดย่อบท คัดย่อจะเป็นส่วนแรกเลยนะคะที่เอาไว้ให้ ผู้อ่านได้คัดกรองก่อนที่จะมาอ่านเอ่อราย ละเอียดจริงในตัวชต Paper นั่นเองนะคะ เดี๋ยวเรามาดูกันก่อนว่าบทคัดย่อเนี่ยควร จะเขียนยังไงนะคะในบทคัดย่อเนี่ยนะคะส่วน ใหญ่แล้วจะประกอบไปด้วยข้อมูลเ่อหลักๆ อยู่ 3 อย่างนะคะคือ 1 ปัญหาและความสำคัญ ของปัญหาหานะคะและ 2 วิธีการแก้ปัญหาของ เราเราทำอย่างไรเราในงานของเราเนี้ยเรา ใช้วิธีอะไรในการแก้ปัญหาที่เรานำเสนอ อยู่นะคะเ่าและ 3 ก็คือผลของปัญหาสรุปผล เลยค่ะว่าปัญหาอ่าหลังจากที่เราเ่อแก้ไข แล้วนะคะเราดำเนินการทำงานเอ่อพยายามแก้ ไขปัญหาที่เราแนะนำไปแล้วเนี่ยผลมันออกมา เป็นอย่างไรนะคะก็มี 3 ส่วนหลักประมาณนี้ นะคะบทคัดย่อจะต้องเขียนแบบกระชับและไม่ ควรเกินครึ่งหน้ากระดาษ A4 นะคะและจะต้อง มีทั้งภาษาไทยและภาษาอังกฤษนะคะดังนั้น เนี่ยในภาษาอังกฤษก็อ่านักศึกษาจะต้องให้ ผู้เชี่ยวชาญทางด้านภาษานะคะเ่าพิจารณา ก่อนที่จะส่งด้วยนะคะทีนี้เรามาดูส่วน ประกอบหลักของชอตเปอร์กันนะคะในเนื้อหา ทั้งหมด 10 หน้ากระดาษมีอะไรบ้างนะคะ อันดับแรกเลยบทนำนะคะบทนำบทนำนะคะเป็นบท ที่กล่าวถึงที่มาที่ไปของปัญญหา หรือว่าความต้องการที่เราจะต้องมาทำงาน โครงการเฉพาะเรื่องนี้เพื่อตอบโจทย์ความ ต้องการนั้นนะคะเราก็จะมากล่าวความเป็นมา นั่นเองนะคะในบทนำนี้นะคะทำไมจะต้องทำ โครงการนี้ปัญหาเราคืออะไรทำไปเพื่ออะไร นะคะพูดง่ายๆคือเพื่อให้ผู้อ่านเข้าใจถึง ประโยชน์และความสำคัญของโครงการว่าทำไม เราถึงต้องทำโครงการนี้นั่นเองนะคะจุด ประสงค์ของโครงการนี้คืออะไรนะคะและตัว ชี้วัดของโครงการนี้คืออะไรนะคะเพราะว่า ทำไมเราต้องใส่ตัวชี้วัดหรือวัตถุประสงค์ ในโครงเ่อในบทนำเพราะว่าเดี๋ยวเราจะไปตอบ ตัววัตถุประสงค์นี้อีกทีในบทสรุปบทสุด ท้ายนั่นเองนะคะดังนั้นก็ต้องระบุให้ชัด เจนว่าอะไรที่เรียกว่าทำสำเร็จแล้วนะคะก็ จะอยู่ในบทนำนะคะจากนั้นก็ต้องบอกขอบเขต ด้วยนะคะบางครบางท่านไม่ได้บอกขอบเขตบาง ท่านพอไม่ได้บอกขอบเขตเนี่ยผู้อ่านก็จะ ไม่สามารถตีเอ่อขอบเขตงานของเราได้นะคะ อาจบางคนอาจจะคิดว่าโหเราทำทั้งระบบเลย หรือเปล่านะคะหรือว่าเราทำทั้งกระบวนการ เลยมั้ยอย่างเงี้ยค่ะเราก็ต้องบอกขอบเขต ด้วยบางคนทำแค่เฉพาะส่วนก็บอกเลยว่าเริ่ม จากส่วนไหนจนถึงส่วนไหนในขอบเขตไหนบ้าง เราควรจะบอกขอบเขตด้วยในบทนำนี้นะคะอัน นั้นก็เป็นส่วนประกอบหลักของบทนำในบทที่ 1  นั่นเองนะคะในบทถัดไปคือบทที่ 2 นะคะบท ที่ 2 ก็คือทฤษฎีหรืองานวิจัยที่เกี่ยว ข้องนะคะในบทนี้เนี่ยจะแสดงให้เห็นถึง องค์ความรู้ต่างๆนะคะหรืองานอื่นๆที่ เกี่ยวข้องกับโครงงานนะคะสิ่งที่ยอดนิยม มากท็อปคิตเลยในการทำบทนี้ก็คือนักศึกษา ส่วนใหญ่จะชอบใส่ทฤษฎีมาอย่างเยอะมากๆ เช่น Network คืออะไร database คืออะไร ไม่เอาอันนี้ไม่เขียนนะคะเราไม่เขียน ทฤษฎีที่เยอะจนเกินไปเพราะผู้อ่านเนี่ย ผู้ประเมินบทความของเราอ่ะมีความรู้อยู่ แล้วในด้านนี้นะคะไม่จำเป็นจะต้องเขียน ทฤษฎีที่ไม่จำเป็นลงไปหรือเยอะเกินไปนะคะ เขียนสั้นๆได้แต่ไม่เยอะและควรเขียนเฉพาะ ทฤษฎีที่เกี่ยวข้องกับงานของเราเท่านั้น ว่าเรานำทฤษฎีไหนมาปรับปรุงใช้ในงานของ เรานะคะสรุปง่ายๆในบทนี้สิ่งที่เราอยาก เห็นก็คือนักศึกษาสามารถสังเคราะห์และ สรุปงานที่มีอยู่แล้วแล้วก็ทฤษฎีที่ เกี่ยวข้องนำมาปรับใช้ในงานของตัวเองได้ นะคะดังนั้นเนี่ยในบทนี้นะคะนอกจากที่จะ กล่าวถึงทฤษฎีที่เกี่ยวข้องหรืองานที่ เอ่อใกล้เคียงกับเราแล้วนะควรจะสามารถ สั่งเคราะห์ออกมานะคะเป็นข้อความได้ด้วย ว่างานที่เราทำอยู่เนี่ยมันสามารถเติม เต็มสิ่งที่มีอยู่แล้วอย่างไรได้บ้างนะคะ แล้วก็งานที่เราทำอยู่ตรงนี้นั้นเนี่ยมี ความเหมือนและความต่างกับงานที่มีอยู่ แล้วอย่างไรได้บ้างนะคะข้อควรระวังเลยนะ คะในบทนี้ส่วนใหญ่ทุกคนก็จะ Copy  pest  ห้ามเลยนะคะการนำข้อความของคนอื่นเข้ามา ใช้ในบทความของเราถือเป็นการขัดลอกงานและ ถือเป็นการทุจริตเลยนะคะอันนี้ยอมรับไม่ ได้ดังนั้นทุกคนห้ามกอปี้เนื้อหาหรือแม้ กระทั่งรูปภาพเข้าใจอยู่ว่ามันเป็นทฤษฎี ที่มีอยู่แล้วเนาะแต่พยายามสรุปด้วยคำพูด ของตัวเองและพยายามที่จะเชื่อมโยงกับงาน ของเราให้ได้นะคะสังเคราะห์ให้ได้ว่าสิ่ง ที่เราเอามาใช้มันเชื่อมโยงกับงานของเรา อย่างไรบ้างส่วนรูปภาพอย่าก๊อปมาเพจ เหมือนกันนะคะรูปภาพนั้นจะต้องวาดเองกับ มือนะคะแล้วก็จะต้องใส่เอ่อตัว Reference  หรือ citation หรือเขาคเรียกว่าการอ้าง อิงที่ถูกต้องด้วยรูปภาพและเนื้อหาที่เอา มาจากงานอื่นจะต้องใส่การอ้างอิงทุกครั้ง นะคะที่มีการกล่าวถึงนะคะถ้าไม่อย่างงั้น เราจะถือว่าเป็นการทุจริตขัดรอกงานของผู้ อื่นค่ะถัดไปนะคะเป็นบทที่ 3 บทที่ 3 ก็ คือเป็นวิธีการทำงานนะคะหรือระเบียบการ วิจัยนะคะบทนี้จะแสดงถึงกระบวนการการทำ งานของเราอย่างละเอียดละเอียดเน้นขีดเส้น ใต้คำว่าละเอียดนะคะบางคนอย่างเช่นพัฒนา แอปพลิเคชันก็บอกว่าอ่ะพัฒนาแอปพลิเคชัน โดยใช้เทคโนโลยีอันนี้เครื่องมืออันนี้ แล้วจบไม่ได้ต้องบอกว่าเราใช้กระบวนการ อะไรในการพัฒนาแอปพลิเคชันเรามีกระบวนการ อะไรบ้างอย่างเช่นมีการเก็บข้อมูลนะคะ หรือ requirement เนาะความต้องการของผู้ ใช้งานจากนั้นนำมาออกแบบดีไซน์ขั้นตอนการ ดีไซน์เป็นยังไงก็ต้องบอกนะคะแล้วก็จาก นั้นนำมาพัฒนาแอปพลิเคชันโดยใช้เครื่อง มืออะไรแบ่งเอ่อ module ของแอปพลิเคชัน เป็นยังไงมีซอฟต์แวร์ Architecture เป็น แบบไหนรายละเอียดต้องบอกให้ชัดเจนมี  database มแล้ว database ออกแบบยังไง ER   diagram ต้องมาแล้วเข้าใจมยคะตัวบทความ นี้นะคะในบทที่ 3 เนี้ยเราจะต้องอธิบาย การทำงานของเราให้ละเอียดที่สุดเป็นขั้น ตอนออกมาให้ได้นะคะจนถึงเอ่อสามารถที่จะ บอกได้ว่าสิ่งที่เราพัฒนาอยู่นั้นเนี่ย มันมีส่วนประกอบอะไรบ้างแล้วแต่ละส่วน ประกอบเรามีรายละเอียดการออกแบบแต่ละส่วน ประกอบยังไงนะคะแต่ถ้าเกิดว่าใครทำเกี่ยว กับการปรับปรุงกระบวนการมันก็จะมีนะคะว่า หลักการปรับปรุงกระบวนการมีอะไรบ้างนำ ทฤษฎีและหลักการที่เรากล่าวถึงในบทที่ 2  มาอธิบายเป็นขั้นตอนในบทนี้ได้เลยโดยเป็น ขั้นตอนของเราเองที่เอามาใช้งานในโครงการ เฉพาะเรื่องของเรานะคะอันนี้เป็นบทที่ 3  นะคะค่ะถัดมาสำหรับบทที่ 4 นะคะหลังจาก ที่เราอธิบายวิธีการพัฒนาหรือกระบวนการทำ งานของเราเรียบร้อยแล้วในบทที่ 3 นะคะใน บทที่ 4 นี้เราจะนำเสนอผลการทดลองหรือผล การพัฒนาของเรานั่นเองนะคะซึ่งการนำเสนอ ก็ทำได้หลากหลายวิธีเลยนะคะก็แล้วแต่ว่า เอ่อการนำเสนอของเราความเหมาะสมจะเป็นยัง ไงนะคะอาจจะขึ้นอยู่กับวัตถุประสงค์ของ การทำงานของเราด้วยในบทที่ 1 ที่เรากล่าว ไว้นะคะเอ่อเราสามารถนำเสนอในเชิงปริมาณ ก็ได้ในเชิงตัวเลขอาจจะวัดมาเป็นตัวเลขก็ ได้นะคะหรือว่าในเชิงคุณภาพก็คืออาจจะวัด มาเป็นในเชิงของฟังก์ชันการทำงานนะคะหรือ ว่าฟังก์ชันอื่นๆที่สามารถอำนวยความสะดวก หรือตอบโจทย์ปัญหาที่เราต้องการจะแก้ไขก็ ได้นะคะหรือว่าจะ Capture รูปภาพหน้าจอนะ คะของตัวเอ่อแอปพลิเคชั่นหรือว่าของ นวัตกรรมที่เราสร้างขึ้นมานะคะก็สามารถ ที่จะถ่ายภาพแคปเจอรูปนะคะแล้วก็นำมานำ เสนอในบทที่ 4 ได้นะคะถัดไปหลังจากบทที่ 4  บทที่ 4 เป็นการนำเสนอผลเฉยๆเนาะแต่ว่า การสรุปเราจะอยู่บทที่ 5 นะคะการสรุปผลนะ คะก็จะเป็นการสรุปผลการทดลองหรือการพัฒนา งานของเรานั่นเองนะคะให้สอดคล้อง คีย์เวิร์ดก็คือให้สอดคล้องกับ วัตถุประสงค์ที่เรากล่าวอยู่ในบทนำนะคะ ว่าบทนำเราต้องการจะแก้ไขปัญหานี้แลละ วัตถุประสงค์คือเพื่ออะไรบทที่ 5 นี่แหละ จะเป็นตัวสรุปว่าเราตอบวัตถุประสงค์ของ เราหรือไม่นะคะเราจะวิเคราะห์ผลจากบทที่ 4  กันนะคะก็คือเพื่อสรุปแนวทางในการปรับ ปรุงพัฒนานะคะพร้อมทั้งสรุปข้อจำกัดต่างๆ หรือแม้กระทั่งงานที่สามารถทำต่อยอดจาก งานนี้ต่อไปได้นะคะบางคนสงสัยว่าบทที่ 5  กับบทคัดย่อต่างกันยังไงนักศึกษาส่วนใหญ่ จะเอ่อมีความสงสัยในเรื่องนี้นะคะคือบท คัดย่อชื่อก็บอกคือคัดย่อก็คือว่าจะจะย่อ ในทุกๆบทเข้าด้วยกันนะคะแต่ว่าบทที่ 5  คือการสรุปผลก็คือสรุปผลการทดลองไม่ต้อง กล่าวมาแล้วว่าเราทำอะไรมาตั้งแต่ต้นนะคะ สรุปเลยว่าผลที่ได้จากบทที่ 4 นั้นเรา สามารถแปลผลออกมาอย่างไรได้บ้างนะคะพร้อม กับข้อจำกัดข้อเสนอแนะและงานที่จะทำต่อ ยอดในอนาคตสามารถเสนอได้อยู่ในบทที่ 5  นี่เองนะคะคือบทสรุปค่ะก็จบไปแล้วสำหรับ คำแนะนำในการเขียนเนื้อหาทั้งหมด 1  +  5  บทนะคะตอนนี้เนี่ยแล้วก็จะมาให้คำแนะนำ อื่นๆที่เกี่ยวข้องนะคะเพื่อคิดว่านัก ศึกษาเนี่ยจะสามารถที่จะเขียนชัเปอรได้ไล ลื่นมากยิ่งขึ้นนะคะโดยอันดับแรกคือ 1 นะ คะเราจะต้องใช้ template จำไว้เลยนะคะเต เรามี template ให้ดาวน์โหลดนะคะจาก เว็บไซต์ของคณะนะคะแล้วก็การเขียนนะคะจะ ต้องตรวจสอบไวยากรณ์ของภาษาให้ถูกต้องการ เว้นวรรคก็ควรจะถูกต้องตามหลักการของภาษา ที่เราเขียนถ้าเราเขียนภาษาไทยก็ต้องใช้ ไวยากรณ์ของภาษาไทยนะคะและระวังให้ดีข้อ ควรระวังก็คือไวยากรณ์ของภาษาไทยอย่าอ่า เอาไปผสมกับไวยากรณ์ของภาษาอังกฤษนะคะ อย่างเช่นบางคนจะใช้คอมม่าภาษาไทยไม่มี คอมม่านะคะแต่ถ้าใครเขียนภาษาอังกฤษมีได้ นะคะภาษาไทยถ้าเรามีการคอมม่าเราอยากคอมม เราใช้เว้นวรรคแทนนะคะและสรรพนามบุรุษที่  2 นะคะเราจะใช้เป็นผู้ทำการศึกษาหรือผู้ ทำการทดลองนะคะใช้ 2  2 สรรพนามนี้เนาะจะ ไม่ใช้ดิฉันไม่ใช้ผมไม่ใช้เรานะและคำที่ มีความหมายเหมือนกันนะคะเราควรเลือกใช้คำ คำเดียวให้สอดคล้องกันทั้งหมดในชอตเปอร์ นั้นนะคะอย่าใช้คำที่หลากหลายนะคะให้ใช้ คำเดียวมันจะได้สอดคล้องกันทางเนื้อหานะ คะและอันนึงเลยที่เราจะเจอประจำก็คือ ปัญหาในเรื่องของการเขียนทัพศัพท์ภาษา อังกฤษเนาะเนื่องจากเราเรียนสายเทคโนโลยี เนาะเราก็จะมีคำศัพท์ภาษาอังกฤษค่อนข้าง เยอะนะคะแนะนำให้ค้นคำศัพท์นะคะในระบบฐาน ข้อข้อมูลนะคะของคำทับศัพท์ราชบัณฑิตยสภา นะคะก็จะมีคำทับศัพท์ให้เราเลือกใช้จะมี วิธีการแนะนำด้วยนะคะสำหรับคำศัพท์ไหนที่ ไม่มีบัญญัติไว้นะคะก็จะมีแนะนำให้เขียน ทับศัพท์อย่างไรก็เข้าไปดูได้ในระบบนั้น เลยนะคะก็จะสามารถใช้คำภาษาอังกฤษได้ อย่างถูกต้องนั่นเองค่ะส่วนการใช้ตัวย่อ นะคะนักศึกษาหลายท่านก็ถามว่าเราจะใช้ตัว ย่อยังไงในบางครบางคำเนี่ยจะยาวมากแล้ว ต้องใช้ตัวย่อนะคะการใช้ตัวย่อก็ง่ายนิด เดียวก็คือครั้งแรกที่เรากล่าวถึงคำนั้น ก็แนะนำตัวย่อเลยนะคะหลังจากนั้นที่เรา ใช้คำนั้นก็ใช้ตัวย่อได้เลยโดยไม่ต้อง กล่าวถึงข้อความเต็มอีกต่อไปนะคะก็สามารถ ใช้กันได้ง่ายๆคือกล่าวครั้งแรกแนะนำ ครั้งแรกจบครั้งถัดไปก็คือใช้ตัวย่อได้ เลยนะคะไม่ต้องเ่อเขียนชื่อเต็มอีกต่อไป ค่ะอันนั้นก็จะเป็นวิธีการใช้ตัวย่อถัดไป การเขียนตารางนะคะหรือรูปภาพนะคะตารางและ รูปภาพทุกอันจะต้องทำเองทั้งหมดอันนี้ เน้นไปแล้วนะคะถ้ากี้ของคนอื่นมานะคะไป ขัดลอกของคนอื่นมาถือว่าเป็นการทุจริตนะ คะถือว่าเป็นการลักขโมยผลงานของคนอื่นดัง นั้นจะต้องทำเองทั้งหมดนะคะและการวางถ้า เกิดว่าตารางและรูปภาพเนี่ยไปอ้างอิงจาก คนอื่นก็ต้องใส่อ้างอิงให้ถูกต้องด้วยนะ คะต้องให้เครดิตของคนที่เรานำเนื้อหามา ด้วยนะคะและการเขียนตารางนั้นนะคะตาราง ทุกอันจะต้องมีหัวข้อนะคะฟังให้ดีนะคะ ชื่อตารางชื่อตารางจะอยู่ด้านบนของตาราง แต่ถ้าเป็นรูปภาพชื่อรูปภาพจะอยู่ด้าน ล่างของรูปภาพนะคะตารางชื่ออยู่ด้านบนรูป ภาพชื่ออยู่ด้านล่างนะคะและทุกภาพทุก ตารางที่ใส่ควรมีเนื้อหารายละเอียดด้วยจะ เห็นเสมอเลยนักศึกษาชอบใส่รูปภาพและตาราง มาแล้วไม่กล่าวถึงอะไรใดๆเลยไม่ได้นะคะจะ ต้องมีคำอธิบายอยู่ในเนื้อหาของตัวชต  Paper เสมออย่างละเอียดด้วยนะคะว่าในรูป ภาพนั้นเนี่ยเราต้องการจะสื่ออะไรมีความ หมายอะไรกล่าวถึงอะไรตารางนั้นเอ่อผลจาก ตารางคืออะไรความหมายอะไรนะคะจะต้องมีให้ ละเอียดชัดเจนห้ามใส่มาลอยๆค่ะแล้วก็ เรื่องอันนึงที่จะต้องเตือนเลยนะคะคือ เรื่องการคัดลอกบทความนะคะเราจะมีการตรวจ จับการขัดลอกบทความเสมอนะคะหลังการส่ง เอ่อตัวช็อตเปอร์ทุกครั้งก่อนการจบการ ศึกษานะคะโดยเราจะส่งเข้าไปในระบบนะคะการ ตรวจสอบการขัดลอกบทความอัตโนมัตินั่นเอง นะคะทุกบทความจะถูกตรวจสอบดังนั้นถ้าใคร ขัดลอกบทความมาเราจะเห็นนะคะแล้วก็อ่าจะ ถือว่าเป็นการทุจริตนะคะส่วนเอ่อบทความใน อ่าของเรานะคะปกติเราจะใช้วิธีการนำเข้า ระบบตรวจสอบอัตโนมัตินะคะแล้วระบบก็จะบอก  similarity นะคะ index ก็คือจะบอกเอ่อ เปอร์เซ็นต์ของความเหมือนกับบทความอื่นๆ นะคะเอ่อในเ่อคณะของเรานั้นนะคะก็จะมี เอ่อเกณฑนะคะของ similarity อยู่ที่ 30%  ถ้าบทความใดก็ตามมี similarity เนี่ยต่ำ กว่า 30% ถือว่าผ่านนะคะเพราะเราเผื่อ เอ่อบทที่ 2 ไงก็คือบทที่อ่านำข้อมูลงาน ของคนอื่นหรือว่าทฤษฎีที่มีอยู่แล้วนะคะ มาเ่อกล่าวในบทที่ 2 ดังนั้นบทนั้นเนี่ย จะค่อนข้างมีตัว similarity ค่อนข้างเยอะ นะคะเราเผื่อเอไว้ให้แล้วนะคะแต่ถ้าใครก็ ตามที่เกิน 30% นั่นหมายถึงมีการขัดลอกบท ความที่มากจนเกินไปนะคะอันนี้ต้องต้องแก้ ไขนะคะหรือว่าถ้าแก้ไขไม่ได้นั่นหมายถึง ว่าเอ่อเราจะตีความว่าคุณขัดลอกผลงานของ ผู้อื่นมาต้องระวังให้ดีนะคะก็เอ่อข้อแนะ นำเลยคือพยายามเขียนด้วยภาษาของตัวเองจะ ดีที่สุดนะคะสำหรับนักศึกษาที่มีคำถาม เพิ่มเติมนะคะสามารถติดต่อเข้ามาได้นะคะ ตามคอนแทคที่ขึ้นไว้ให้บนหน้าจอได้เลยนะ คะก็จะมีเจ้าหน้าที่คอยเ่อตอบคำถามเกี่ยว กับชตเปอร์อำนวยความสะดวกในการทำช Paper  ให้กับนักศึกษานะคะก็ติดต่อกันเข้ามาได้ ตามคอคที่ขึ้นทวหน้าจอค่ะ emp  ballet   Digital  transformation สร้างศักยภาพ สู่ยุคดิจิตัลอย่างไร้ขีดจำกัดคณะ เทคโนโลยีสารสนเทศมหาวิทยาลัยเทคโนโลยี พระจอมเกล้า ธนบุรี e

#### docx

In [35]:
example_doc = all_docs[3]
# display(example_doc)
print(len(example_doc))

for doc in example_doc:
    doc_metadata = doc.metadata
    doc_content = doc.page_content
    display(doc_metadata)
    display(Markdown(doc_content))

2


{'source': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/Template-short-Paper-3-1.docx',
 'category_depth': 0,
 'file_directory': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads',
 'filename': 'Template-short-Paper-3-1.docx',
 'last_modified': '2025-06-10T22:25:38',
 'page_number': 1,
 'languages': ['tha'],
 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'emphasized_text_contents': ['(',
  '(',
  'กรุณาเลือกตำแหน่งทางวิชาการของอาจารย์ที่ปรึกษา',
  'กรุณาเลือกตำแหน่งทางวิชาการของอาจารย์ที่ปรึกษา',
  'กรณีตำแหน่ง',
  'กรณีตำแหน่ง',
  'ทางวิชาการชอง',
  'ทางวิชาการชอง',
  'อาจารย์ที่ปรึกษา',
  'อาจารย์ที่ปรึกษา',
  'เป็น',
  'เป็น',
  '“',
  '“',
  'ดร.',
  'ดร.',
  '”',
  '”',
  'ให้เลือก',
  'ให้เลือก',
  '“',
  '“',
  'อาจารย์',
  'อาจารย์',
  '”',
  '”',
  ')',
  ')'],
 'emphasized_text_tags': ['b',
  'i',
  'b',
  'i',
  'b',
  'i',
  'b',
  'i',
  'b',
  'i',
  'b',
  'i',
  'b',
  'i',
  

ชื่อบทความ

ชื่อนักศึกษา*1  และ ชื่ออาจารย์2  

(ชื่อนักศึกษาและชื่ออาจารย์ใส่เฉพาะชื่อ-นามสกุลเท่านั้น)

มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี 126 ถนนประชาอุทิศ แขวงบางมด เขตทุ่งครุ กรุงเทพฯ 10140 

บทคัดย่อ 

บทความนี้กล่าวถึงรูปแบบและวิธีการส่งบทความฉบับสมบูรณ์ เพื่อตีพิมพ์ในวารสารไทยวิจัยดำเนินงาน ผู้ส่งบทความ จะต้องยึดรูปแบบตามบทความนี้อย่างเคร่งครัด บทความใดที่รูปแบบไม่ถูกต้องอาจถูกส่งคืนและไม่รับพิจารณาต่อไป บทคัดย่อ ต้องมีทั้งภาษาไทยและภาษาอังกฤษ แต่ละภาษาควรมีเพียงย่อหน้าเดียว และความยาวประมาณ 250 คำ อนึ่งบทคัดย่อภาษาไทย ต้องเขียนด้วยภาษาไทย ถ้ามีความจำเป็นต้องใช้คำเฉพาะภาษาอังกฤษ ให้เขียนทับศัพท์ภาษาอังกฤษ ด้วยภาษาไทย    

คำสำคัญ: ภาษาไทย จัดรูปแบบบทความ กระจายแบบไทย, รูปแบบตัวอักษร TH SarabunPSK  ชื่อบทความใช้อักษรขนาด 18 แบบหนา, ชื่อผู้แต่งและรายละเอียดใช้ตัวอักษรขนาด 14 แบบหนา, เนื้อหาบทคัดย่อใช้อักษรขนาด 14 แบบธรรมดา, หัวเรื่อง (บทคัดย่อ) ใช้ตัวอักษรขนาด 16 แบบหนา  ทั้งนี้คำสำคัญ ให้เรียงลำดับ ก-ฮ  ควรมีคำสำคัญ 5 คำสำคัญขึ้นไป

-----------------------------------------------------------------------------------------------------------------------------------------------

Author’s Email: idstudent@sit.kmutt.ac.th    (ใส่อีเมลของนักศึกษา)

1 นักศึกษาปริญญาโท สาขาวิชาเทคโนโลยีสารสนเทศ/สาขาวิชาระบบสารสนเทศทางธุรกิจ/สาขาวิชาวิศวกรรมซอฟต์แวร์/สาขาวิชาระบบสารสนเทศทางธุรกิจดิจิทัล/วิศวกรรมซอฟต์แวร์เพื่อวิทยาการข้อมูล คณะเทคโนโลยีสารสนเทศ มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี

 (เลือกสาขาวิชาที่กำลังศึกษาอยู่)

2 อาจารย์/ผู้ช่วยศาสตราจารย์/รองศาสตราจารย์ คณะเทคโนโลยีสารสนเทศ มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี

 (กรุณาเลือกตำแหน่งทางวิชาการของอาจารย์ที่ปรึกษา กรณีตำแหน่งทางวิชาการชองอาจารย์ที่ปรึกษาเป็น “ดร.” ให้เลือก “อาจารย์”)





{'source': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/Template-short-Paper-3-1.docx',
 'category_depth': 0,
 'file_directory': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads',
 'filename': 'Template-short-Paper-3-1.docx',
 'last_modified': '2025-06-10T22:25:38',
 'page_number': 2,
 'languages': ['tha'],
 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'emphasized_text_contents': ['Minor R', 'evisions'],
 'emphasized_text_tags': ['b', 'b'],
 'link_texts': ['คลิ๊กดูคู่มือการเขียนบรรณานุกรม'],
 'link_urls': ['https://www2.sit.kmutt.ac.th/isreport/New/APA-Referencing-Guide.pdf'],
 'links': [{'text': 'คลิ๊กดูคู่มือการเขียนบรรณานุกรม',
   'url': 'https://www2.sit.kmutt.ac.th/isreport/New/APA-Referencing-Guide.pdf',
   'start_index': 0}]}

Short Paper’s Title

Student name*1 and Lecturer name2  

(Student name and Lecturer name, enter only first and last name only)

King Mongkut’s University of Technology Thonburi  

126 PrachaUthit Rd., BangMod, ThungKru, Bangkok 10140, Thailand 

Abstract 

This article describes a submission procedure and a format of the manuscript for paper publication. The authors are required to strictly follow the guideline provided here, otherwise the manuscript will be returned without publication consideration, and you must register the course again next term. A good abstract should contain only one paragraph. Both Thai and English abstracts are required. The length of each abstract should not exceed 250 words.  

Keywords: manuscript format, font size 14 point, font style TH SarabunPSK   

The keywords sort by A-Z; 5 or more words

____________________________          

Author’s Email: idstudent@sit.kmutt.ac.th  (Enter your student email)

1. Postgraduate Student, Information Technology Program/Business Information System Program/ Software Engineering Program/ Digital Business Information System Program/Software Engineering for Data Science Program, School of Information Technology, King Mongkut’s University of Technology Thonburi    (Select your program)

2. Lecturer/Assistant Professor/ Associate Professor, School of Information Technology, King Mongkut’s University of Technology Thonburi      (Please select the academic position of the advisor, if your advisor is “Dr.”, please select “Lecturer”)

บทนำ 

บทความประกอบด้วยส่วนต่างๆ ตามลำดับต่อไปนี้ คือ ชื่อเรื่องภาษาไทย ชื่อเรื่องภาษาอังกฤษ  ชื่อผู้เขียน บทความ สถาบัน ที่อยู่สถาบันอย่างละเอียด บทคัดย่อ ภาษาไทย และภาษาอังกฤษ (ต้องมี) เนื้อเรื่องแบ่งเป็น บทนำ แนวคิด ทฤษฎี และงานวิจัยที่เกี่ยวข้อง วิธีดำเนินการ วิจัย ผลการวิจัย สรุปผลการวิจัย กิตติกรรมประกาศ (ถ้ามี) เอกสารอ้างอิง ภาคผนวก (ถ้ามี)  

คำแนะนำการเขียนและพิมพ์

รูปแบบและการตั้งค่ากระดาษ 

  บทความที่เสนอต้องพิมพ์เป็นภาษาไทยหรือภาษาอังกฤษ ตามรูปแบบที่กำหนด ควรมีความยาวไม่เกิน 10 หน้ากระดาษ ขนาดกระดาษ A4 (21.0 ซม. x 29.7 ซม.) โดยจัดหน้าเป็นแบบ 2 คอลัมน์ ตามรูปแบบบทความนี้ ขนาดของคอลัมน์ 3.04 ซม. ระยะห่างระหว่างคอลัมน์เป็น 29 มม. ระยะห่างจากขอบซ้ายและขวา เป็น 0.98 นิ้ว ขอบ บนเป็น 1.17 นิ้ว ขอบล่างเป็น 0.98  มม.ตั้งค่าหัวกระดาษที่ 0.39 นิ้ว ท้ายกระดาษ 0.2 นิ้ว ตั้งย่อหน้าไว้ที่ 10 มม. ให้ พิมพ์โดยไม่เว้นบรรทัด แต่ให้เว้น 1 บรรทัดระหว่างหัวข้อ หลักทุกครั้ง ส่วนระหว่างหัวข้อหลักกับหัวข้อรองและหัวข้อ รองกับหัวข้อย่อย ให้เว้น Space เป็น before = 6 pt และ after = 0 pt  โดยจะต้องพิมพ์ให้เต็มคอลัมน์ก่อนที่จะขึ้น คอลัมน์ใหม่หรือขึ้นหน้าใหม่ ห้ามเว้นที่เหลือไว้ว่างเปล่า

 การลำดับหัวข้อในส่วนของเนื้อเรื่อง ให้ใส่เลขกำกับ โดยให้บทนำเป็นหัวข้อหมายเลข 1 และหากมีการแบ่งหัวข้อ รอง หรือหัวข้อย่อย ให้ใช้เลขระบบทศนิยมกำกับหัวข้อย่อย เช่น 2.1 2.2 2.1.1 2.2.1 เป็นต้น

ชนิดและขนาดตัวอักษร

 พิมพ์บทความภาษาไทยหรือภาษาอังกฤษด้วยตัวอักษร รูปแบบ “TH SarabunPSK” ทั้งหมด ชื่อเรื่องบทความ ใช้ ตัวอักษรแบบหนาขนาด 18  ชื่อผู้เขียน สถาบัน ใช้ตัวอักษร แบบหนาขนาด 14  ชื่อหัวข้อหลักใช้ตัวอักษรแบบหนาขนาด 16 และชื่อหัวข้อรองและหัวข้อย่อยใช้ตัวอักษรแบบหนาขนาด 14 บทคัดย่อและเนื้อความต่างๆ ใช้ตัวอักษรแบบ ธรรมดาขนาด 14 

ชื่อเรื่อง ชื่อผู้แต่ง และชื่อหัวข้อ 

การพิมพ์ชื่อเรื่อง ให้วางไว้ตำแหน่งกลาง หน้ากระดาษ แบบคอลัมน์เดี่ยว เริ่มจากชื่อเรื่องภาษาไทย ขึ้นบรรทัดใหม่เป็นชื่อเรื่องภาษาอังกฤษ ชื่อผู้เขียนและ สถาบันให้พิมพ์ไว้ใต้ชื่อเรื่องและอยู่กลางหน้ากระดาษ แบบ คอลัมน์เดี่ยว ระบุที่อยู่ของที่ทำงานอย่างละเอียด ระบุ หมายเลขโทรศัพท์ ระบุหมายเลขโทรสาร (ถ้ามี) ระบุอีเมล ไม่ต้องระบุตำแหน่งทางวิชาการหรือสถานะของนักศึกษาใด ๆ ทั้งสิ้น ชื่อหัวข้อหลัก และหัวข้อรอง ให้วางตำแหน่งชิดขอบ ซ้ายยกเว้นหัวข้อย่อยให้วางในตำแหน่งเดียวกับย่อหน้า  ตัวอย่างเช่น 

การจัดทำรูปภาพ

 รูปภาพจะต้องมีความกว้างไม่เกิน 3.04 นิ้ว เพื่อให้ลง ในหนึ่งคอลัมน์ได้ หรือในกรณีจำเป็นจริงๆ เพื่อรักษา รายละเอียดในภาพอาจยอมให้มีความกว้างได้เต็ม หน้ากระดาษ  

ตัวอักษรทั้งหมดในรูปภาพ จะต้องมีขนาดใหญ่ สามารถอ่านได้สะดวก

  รูปภาพทุกรูปจะต้องมีหมายเลขและคำบรรยายได้ ภาพ หมายเลขและคำบรรยายรวมกันแล้วควรจะมีความยาว ไม่เกิน 2 บรรทัด

คำบรรยายใต้ภาพ ห้ามใช้คำว่า “แสดง” เช่น ห้าม เขียนว่า”รูปที่ 1 แสดงความสัมพันธ์...” 

ที่ถูกต้องควรเป็น “รูปที่ 1 ความสัมพันธ์ระหว่าง...” รูปลายเส้นต้องเป็นเส้นหมึกดำ ส่วนรูปถ่ายควรเป็น รูปขาวดำที่มีความคมชัด รูปสีอนุโลมให้ได้ รูปภาพควรมี รายละเอียดเท่าที่จำเป็นเท่านั้น เช่น ภาพถ่ายรูปคลื่นจาก ออสซิลโลสโคป ที่ปรากฏให้เห็นเฉพาะจอภาพ เป็นต้น และ เพื่อความสวยงามให้เว้นบรรทัดเหนือรูปภาพ 1 บรรทัด และ เว้นใต้คำบรรยายรูปภาพ 1 บรรทัด และกำหนดให้รูปภาพ อยู่ตรงกลาง

การหมุนของวัตถุทรงกระบอกในวัตถุทรงกระบอกอีก ชิ้นหนึ่ง (TH SarabunPSK 12 เอียง)

การเขียนสมการ

สมการทุกสมการต้องมีหมายเลขกำกับอยู่ภายใน วงเล็บ และเรียงลำดับที่ถูกต้อง ตำแหน่งของหมายเลข สมการต้องอยู่ชิดขอบด้านขวาของคอลัมน์ ดังตัวอย่างนี้                                   

a+b=c                (1)

ส่วนตัวอักษรในสมการต่างๆ ให้ใช้ตัวอักษร Times New Roman ขนาด 12 ตัวสัญลักษณ์ให้ใช้ Math Type ใน การเขียนสมการ

 การจัดทำตาราง 

ตัวอักษรในตารางต้องไม่เล็กกว่าตัวอักษรในเนื้อเรื่อง ควรตีเส้นกรอบตารางด้วยหมึกดำให้ชัดเจน ตารางทุกตาราง

ต้องมีหมายเลขและคำบรรยายกำกับเหนือตาราง หมายเลข กำกับและคำบรรยายนี้รวมกันแล้วควรมีความยาวไม่เกิน 2    

บรรทัดในคำบรรยายเหนือตารางห้ามใช้คำว่า “แสดง” เช่นเดียวกับกรณีรูปภาพ เพื่อความสวยงาม ให้เว้นบรรทัด เหนือคำบรรยายตาราง 1 บรรทัด และเว้นบรรทัดใต้ตาราง 1 บรรทัด  หากตารางมีขนาดใหญ่ อนุญาตให้มีความกว้างได้ เต็มหน้ากระดาษเพื่อรักษารายละเอียดในตาราง  

ตัวอย่างการเขียนตาราง (TH SarabunPSK 12 เอียง)  

การอ้างอิงและเอกสารอ้างอิง  

การอ้างอิงเอกสารอ้างอิงจะใช้หมายเลขอาราบิคที่อยู่ ในวงเล็บสี่เหลี่ยม (Square brackets) เช่น [2] ดังแสดง ตัวอย่างในหัวข้อ เอกสารอ้างอิง โดยเรียงลำดับ [1], [2], … ต้องเรียงลำดับหมายเลขอ้างอิงจากหมายเลขน้อยไปสู่ หมายเลขมากให้ถูกต้อง การอ้างอิงหมายเลขที่มีลำดับติดต่อ กับให้ใช้รูปแบบดังนี้ [1-5] ให้พิมพ์ตามรูปแบบมาตรฐาน IEEE โดยต้องระบุชื่อบทความที่อ้างอิงให้ชัดเจน ให้จัด รายการอ้างอิงให้อยู่ในแนวตรงตามตัวอย่าง โดยเว้นระยะ จากขอบซ้ายให้ตรงกันทุกรายการ

ความยาวของบทความ 

เมื่อรวมทุกส่วนแล้ว แต่ละบทความควรมีความยาว ไม่น้อยกว่า 8 หน้าและไม่เกิน 10 หน้ากระดาษ เพื่อความรวดเร็วในการประเมิน บทความเพื่อตีพิมพ์ บทความที่มีความยาวมากอาจได้รับการ ประเมินช้าและส่งผลให้ได้รับการตีพิมพ์ช้าด้วย

กรณีบทความภาษาอังกฤษ 

ผู้พิมพ์บทความเป็นภาษาอังกฤษ ขอให้จัดรูปแบบ ตามที่กำหนดไว้นี้เช่นกัน สำหรับข้อความภาษาอังกฤษที่อยู่ ในวงเล็บให้ตัวอักษรแรกของคำพิมพ์ด้วยตัวพิมพ์ใหญ่ หากมี หลายพยางค์กำหนดให้อักษรตัวแรกของคำแรกเท่านั้นที่ พิมพ์ด้วยตัวพิมพ์ใหญ่ ยกเว้นคำศัพท์เฉพาะ ชื่อเฉพาะ เช่น  (Minor Revisions)  นอกจากนี้ บทความภาษาอังกฤษต้องมี บทคัดย่อและชื่อเรื่องภาษาไทยด้วย

สรุป 

ผู้เขียนบทความกรุณาตรวจบทความอย่างรอบคอบ โดยใช้เวลาอย่างเพียงพอก่อนส่งให้ผู้ทรงคุณวุฒิพิจารณา เพราะจะทำให้บทความของท่านมีคุณภาพสูงและผ่านการ พิจารณาได้ง่ายขึ้น  

กิตติกรรมประกาศ (ถ้ามี) 

ขอขอบคุณผู้เขียนบทความทุกท่านที่ให้ความร่วมมือ และรักษาระเบียบการเขียนบทความอย่างเคร่งครัด

เอกสารอ้างอิง 

โปรดศึกษาจากคู่มือการเขียนบรรณานุกรม

คลิ๊กดูคู่มือการเขียนบรรณานุกรม

ภาคผนวก (ถ้ามี)  

กรณีรูปภาพหรือตารางมีขนาดใหญ่มาก เมื่อย่อแล้วขนาดเล็กเกินไปจนไม่สามารถมาอ่านเข้าใจได้ อนุญาตให้ ขยายขนาดจนเต็ม 2 คอลัมน์ได้ ทั้งนี้ชื่อภาพหรือตารางให้ จัดรูปแบบเป็น 1 คอลัมน์ตามภาพด้วย โดยที่เนื้อหาก่อนและหลังรูปภาพ/ตารางยังคงอยู่ในรูปแบบ 2 คอลัมน์  



In [66]:
example_doc = all_docs[-1]
# display(example_doc)
print(len(example_doc))

for doc in example_doc:
    doc_metadata = doc.metadata
    doc_content = doc.page_content
    display(doc_metadata)
    display(Markdown(doc_content))

1


{'source': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/การโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่อง-BIS.docx',
 'category_depth': 0,
 'file_directory': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads',
 'filename': 'การโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่อง-BIS.docx',
 'last_modified': '2025-06-10T22:23:21',
 'languages': ['tha'],
 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'emphasized_text_contents': ['ข้อตกลงว่าด้วยการโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่อง'],
 'emphasized_text_tags': ['b'],
 'parent_id': '5dfb4dfd9b629bf45558ccfac34a78fb'}

AA-U-F09-05 R01

มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี

ข้อตกลงว่าด้วยการโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่อง

						วันที่………….เดือน………….พ.ศ…………..

	ข้าพเจ้า (นาย/นาง/นางสาว)…………………………………รหัสประจำตัว…………….

เป็นนักศึกษาของมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี   ระดับปริญญา	        โท	         เอก

หลักสูตร วิทยาศาสตรมหาบัณฑิต  สาขาวิชา  ระบบสารสนเทศทางธุรกิจ  คณะ เทคโนโลยีสารสนเทศ

อยู่บ้านเลขที่………………….ตรอก/ซอย…………………………ถนน…………………………..

ตำบล/แขวง………………………….อำเภอ/เขต……………………….จังหวัด…………………...

รหัสไปรษณีย์…………………………………เป็น “ผู้โอน” ขอโอนลิขสิทธิ์การศึกษาโครงการเฉพาะเรื่องให้ไว้กับมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี โดยมี ผศ.ดร.ณรงค์ฤทธิ์ วราภรณ์ 

ตำแหน่ง…….คณบดีคณะเทคโนโลยีสารสนเทศ…………เป็นตัวแทน “ผู้รับโอน” สิทธิในทรัพย์สินทางปัญญาและมีข้อตกลงดังนี้

	1.  ข้าพเจ้าได้จัดทำการศึกษาโครงการเฉพาะเรื่อง เรื่อง……………………………………

……………………………………………………………………………………………………….

ซึ่งอยู่ในความควบคุมของ…………………………………………………………………………...

ตามมาตราพระราชบัญญัติลิขสิทธิ์ พ.ศ.  2537 และถือว่าเป็นส่วนหนึ่งของการศึกษาตามหลักสูตร

ของมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี

ข้าพเจ้าตกลงโอนลิขสิทธิ์จากผลงานทั้งหมดที่เกิดขึ้นจากการสร้างสรรค์ของข้าพเจ้า

ในการศึกษาโครงการเฉพาะเรื่องให้กับมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี ตลอดอายุแห่งการคุ้มครองลิขสิทธิ์ตามพระราชบัญญัติลิขสิทธิ์ พ.ศ. 2537 ตั้งแต่วันที่ได้รับอนุมัติโครงร่างการศึกษาโครงการเฉพาะเรื่องจากมหาวิทยาลัย

ในกรณีที่ข้าพเจ้าประสงค์จะนำการศึกษาโครงการเฉพาะเรื่องไปใช้ในการเผยแพร่ใน

สื่อใด ๆ ก็ตาม ข้าพเจ้าจะต้องระบุว่าการศึกษาโครงการเฉพาะเรื่องเป็นผลงานของมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรีทุกครั้งที่มีการเผยแพร่

ในกรณีที่ข้าพเจ้าประสงค์จะนำการศึกษาโครงการเฉพาะเรื่องไปเผยแพร่หรืออนุญาต

ให้ผู้อื่นทำซ้ำหรือดัดแปลงหรือเผยแพร่ต่อสาธารณชนหรือกระทำการอื่นใด ตามพระราชบัญญัติลิขสิทธิ์ พ.ศ. 2537 โดยมีค่าตอบแทนในเชิงธุรกิจ ข้าพเจ้าจะกระทำได้เมื่อได้รับความยินยอมเป็นลายลักษณ์อักษรจากมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรีก่อน

ในกรณีที่ข้าพเจ้าประสงค์จะนำข้อมูลจากการศึกษาโครงการเฉพาะเรื่องไปประดิษฐ์

หรือพัฒนาต่อยอดเป็นสิ่งประดิษฐ์หรืองานทรัพย์สินทางปัญญาประเภทอื่น ภายในระยะเวลา (10) ปีนับจากวันลงนามในข้อตกลงฉบับนี้ ข้าพเจ้าจะกระทำได้เมื่อได้รับความยินยอมเป็นลายลักษณ์อักษรจากมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี และมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรีมีสิทธิในทรัพย์สินทางปัญญานั้น พร้อมกับได้รับชำระค่าตอบแทนการอนุญาตให้ใช้สิทธิดังกล่าว รวมถึงการจัดสรรผลประโยชน์อันพึงเกิดขึ้นจากส่วนใดส่วนหนึ่งหรือทั้งหมดของ

การศึกษาโครงการเฉพาะเรื่องในอนาคต โดยให้เป็นไปตามระเบียบสถาบันเทคโนโลยีพระจอมเกล้าธนบุรี ว่าด้วยการบริหารผลประโยชน์อันเกิดจากทรัพย์สินทางปัญญา พ.ศ. 2538

ในกรณีที่มีผลประโยชน์เกิดขึ้นจากการศึกษาโครงการเฉพาะเรื่องหรืองานทรัพย์สิน

ทางปัญญาอื่นที่ข้าพเจ้าทำขึ้นโดยมีมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรีเป็นเจ้าของ ข้าพเจ้าจะมีสิทธิ์ได้รับการจัดสรรผลประโยชน์อันเกิดจากทรัพย์สินทางปัญญาดังกล่าวตามอัตราที่กำหนดไว้ในระเบียบสถาบันเทคโนโลยีพระจอมเกล้าธนบุรี ว่าด้วยการบริหารผลประโยชน์อันเกิดจากทรัพย์สินทางปัญญา พ.ศ. 2538

ลงชื่อ………………………………..ผู้โอนลิขสิทธิ์	 	ลงชื่อ………………………………..พยาน

         (                                             )			             ( นางถิรดา  ยุกตะนันทน์ )             

ลงชื่อ………………………………..ผู้รับโอนลิขสิทธิ์	ลงชื่อ………………………………..พยาน

        (  ผศ.ดร.ณรงค์ฤทธิ์ วราภรณ์)                                             ( นางสาวอาภรณ์ เชี่ยวชาญเกษตร)



### split into chunks

In [142]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                            chunk_overlap=200,)
                            # length_function=len,  # how length is calculated; default is `len`
                            # separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],)  # custom split hierarchy)

In [143]:
all_splits = []
all_splits.extend(text_splitter.split_documents(all_docs))

In [144]:
len(all_splits)

76

In [146]:
all_splits[0]

Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-18T12:50:12+07:00', 'author': 'WALAIPUN PORNWIROON', 'moddate': '2025-05-18T12:50:12+07:00', 'source': '/Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='ขั้นตอนที่ต้องปฏิบัติก่อนถึงวันสอบ Short Paper ขั้นสุดท้าย (เฉพาะนักศึกษารหัสนำหน้า 63XXXXXXXXX \nขึ้นไป) \n            1.นักศึกษาต้องแก้ไขตามคำแนะนำของอาจารย์ที่ปรึกษาและกรรมการภายในคณะฯ ให้เสร็จสิ้นเรียบร้อย      \nทั้งนี้ระบบจะดึงบทความ Short Paper ฉบับล่าสุดให้กับผู้ทรงคุณวุฒิภายนอกอัตโนมัติ \n2. ดำเนินการ Upload คลิป VDO บรรยายผลการศึกษา Short Paper ความยาวประมาณ ไม่เกิน 5 นาที      \nไฟล์ PowerPoint และบทความ Short Paper ฉบับสมบูรณ์ที่ผ่านจากกรรมการภายในคณะ 2 ท่านเรียบร้อยแล้ว ให้ \nUpload link ใน Google Drive เท่านั้น ไม่อนุญาต Upload link ใน YouTube โดยนำ link ที่จัดเ

### Page Number

In [147]:
from langchain_core.documents.base import Document

for doc in tqdm(all_splits):
    source = doc.metadata["source"]
    print(f"Processing source: {source}")
    print("---")
    file_ext = os.path.splitext(source)[1].lower()
    print(f"file extension is: {file_ext}")
    if file_ext == ".docx":
        filetype = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
    elif file_ext == ".pdf":
        filetype = "application/pdf"
    elif file_ext == ".txt":
        filetype = "text/plain"
    else:
        raise ValueError(f"Unsupported file extension: {ext}")
    print(f"filetype for api call: {filetype}")
    print("---")
    
    if isinstance(doc, Document):
        doc_metadata = doc.metadata
        doc_content = doc.page_content
    else:
        raise KeyError("doc is not and instance of langchain_core.documents.base.Document")
        
    if "page_label" in doc_metadata or "page_number" in doc_metadata:
        if file_ext == '.pdf':
            page_number = doc_metadata["page_label"]
        elif file_ext == '.docx' or file_ext == '.doc':
            page_number = doc_metadata["page_number"]
    else:
        # have only a single page
        page_number = 1
    if file_ext == '.txt':
        page_numnber = None
    print(f"page number of this doc is: {page_number}")
    print("---")
    print()

  0%|          | 0/76 [00:00<?, ?it/s]

Processing source: /Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf
---
file extension is: .pdf
filetype for api call: application/pdf
---
page number of this doc is: 1
---

Processing source: /Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf
---
file extension is: .pdf
filetype for api call: application/pdf
---
page number of this doc is: 1
---

Processing source: /Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf
---
file extension is: .pdf
filetype for api call: application/pdf
---
page number of this doc is: 1
---

Processing source: /Users/picetrp/Documents/Learn/KMUTT/MS/projects/simple_rag/data/uploads/รายละเอียดในการสอบ_short_paper.pdf
---
file extension is: .pdf
filetype for api call: application/pdf
---
page number of this doc is: 2
---

Processing source: /Users/picetrp/Documents/Learn/KMUTT/MS/projects/

# Embedding

### Setup gemini

In [445]:
from google import genai

gemini_client = genai.Client()

In [288]:
for m in gemini_client.models.list():
    if 'embedContent' in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [343]:
from google.genai import types
from google.genai.client import Client
from typing import List, Union

# Function to convert text to embeddings, can also do batch embeddings
def make_embed_text_fn(client: Client,
                       text: Union[str, List[str]], 
                       model_id: str = "models/text-embedding-004",
                       task_type: str ="retrieval_document") -> Union[List[float], List[List[float]]]:
    response = client.models.embed_content(
        model=model_id,
        contents=text,
        config=types.EmbedContentConfig(
          task_type=task_type,
        )
    )
    return [embedding.values for embedding in response.embeddings]

In [344]:
text = all_splits[0].page_content
embedding = make_embed_text_fn(gemini_client, text, 
                               model_id="models/text-embedding-004", 
                               task_type="retrieval_document")
len(embedding)

1

In [286]:
texts = [i.page_content for i in all_splits[:8]]
embeddings = make_embed_text_fn(gemini_client, texts, 
                               model_id="models/text-embedding-004", 
                               task_type="retrieval_document")
len(embeddings)

8

In [252]:
texts = [i.page_content for i in all_splits[:8]]
metadatas = [i.metadata for i in all_splits[:8]]

### Batch & Rate limit

In [289]:
import time
import random
from typing import List
from google.genai import types
from google.genai.client import Client
from google.api_core.exceptions import ResourceExhausted

# --- Constants ---
MAX_REQUESTS_PER_SECOND = 25
MIN_DELAY_BETWEEN_REQUESTS = 1.0 / MAX_REQUESTS_PER_SECOND  # 0.04s
DEFAULT_BATCH_SIZE = 128  # max recommended

# --- Helper: Chunk data ---
def chunked(texts: List[str], batch_size: int) -> List[List[str]]:
    return [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]

# --- Helper: Backoff for errors (rate limit, etc.) ---
def embed_with_backoff(client: Client,
                       texts: List[str],
                       model_id: str,
                       task_type: str,
                       max_retries: int = 5):
    for attempt in range(max_retries):
        try:
            return client.models.embed_content(
                model=model_id,
                contents=texts,
                config=types.EmbedContentConfig(task_type=task_type)
            )
        except ResourceExhausted as e:
            wait = (2 ** attempt) + random.uniform(0, 1)
            print(f"Rate limit hit. Backing off {wait:.2f}s... (attempt {attempt + 1})")
            time.sleep(wait)
        except Exception as e:
            print(f"Unexpected error: {e}")
            raise
    raise RuntimeError("Max retries exceeded due to rate limits or other failures.")

# --- Main Function: Embed all with strict rate limiting ---
def embed_texts_strict_rate(client: Client,
                            texts: List[str],
                            model_id: str = "models/text-embedding-004",
                            task_type: str = "retrieval_document",
                            batch_size: int = DEFAULT_BATCH_SIZE) -> List[List[float]]:
    
    all_embeddings = []
    batches = chunked(texts, batch_size)

    for i, batch in tqdm(enumerate(batches)):
        start_time = time.time()
        
        response = embed_with_backoff(client, batch, model_id, task_type)
        all_embeddings.extend([embedding.values for embedding in response.embeddings])

        elapsed = time.time() - start_time
        sleep_time = max(0, MIN_DELAY_BETWEEN_REQUESTS - elapsed)
        if sleep_time > 0:
            time.sleep(sleep_time)

        print(f"Processed batch {i+1}/{len(batches)}")

    return all_embeddings

In [293]:
# --- Constants ---
MAX_REQUESTS_PER_SECOND = 25
MIN_DELAY_BETWEEN_REQUESTS = 1.0 / MAX_REQUESTS_PER_SECOND  # 0.04s
DEFAULT_BATCH_SIZE = 64  # max recommended

mock_text_chunks = []
for i in range(20):
    mock_text_chunks.extend(all_splits)
mock_text_chunks = [i.page_content for i in mock_text_chunks]
batches = chunked(mock_text_chunks, DEFAULT_BATCH_SIZE)

all_embeddings = embed_texts_strict_rate(gemini_client, 
                                         texts=mock_text_chunks,
                                         model_id = "models/text-embedding-004",
                                         task_type = "retrieval_document",
                                         batch_size = DEFAULT_BATCH_SIZE)

0it [00:00, ?it/s]

Processed batch 1/24
Processed batch 2/24
Processed batch 3/24
Processed batch 4/24
Processed batch 5/24
Processed batch 6/24
Processed batch 7/24
Processed batch 8/24
Processed batch 9/24
Processed batch 10/24
Processed batch 11/24
Processed batch 12/24
Processed batch 13/24
Processed batch 14/24
Processed batch 15/24
Processed batch 16/24
Processed batch 17/24
Processed batch 18/24
Processed batch 19/24
Processed batch 20/24
Processed batch 21/24
Processed batch 22/24
Processed batch 23/24
Processed batch 24/24


In [295]:
len(all_embeddings[0])

768

In [294]:
len(all_embeddings)

1520

In [6]:
# import os
# from huggingface_hub import login
# login(token=settings.huggingface_token)
# encoder = SentenceTransformer("all-MiniLM-L6-v2")

# Vector Database - Qdrant

### Trial Setup

In [151]:
from qdrant_client import models, QdrantClient
# from sentence_transformers import SentenceTransformer

In [209]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(url=settings.qdrant_cloud_url, api_key=settings.qdrant_cloud_api_key)

In [334]:
from qdrant_client.models import Distance, VectorParams

def create_collection(client: QdrantClient,
                      collection_name: str = "demo_collection",
                      vector_size: int = 384, 
                      distance: Distance = Distance.COSINE, 
                      **kwargs) -> None:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=vector_size, 
            distance=distance
        ),
    )
    return 

def delete_collection(client: QdrantClient,
                      collection_name: str = "demo_collection",
                      **kwargs) -> None:
    client.delete_collection(
        collection_name=collection_name,
    )

def delete_by_ids(client:QdrantClient,
                  ids: List[Union[int, str]],
                 collection_name: str = "demo_collection") -> None:
    client.delete(
        collection_name=collection_name,
        points_selector=models.PointIdsList(
            points=ids,
        ),
    )

def get_all_point_ids(client: QdrantClient, collection_name: str) -> List[Union[int, str]]:
    all_ids = []
    scroll_cursor = None

    while True:
        response = client.scroll(
            collection_name=collection_name,
            scroll_filter=None,
            with_payload=False,
            with_vectors=False,
            # limit=100,  # set max limit to reduce number of requests
            offset=scroll_cursor
        )
        points, scroll_cursor = response

        all_ids.extend([point.id for point in points])

        if scroll_cursor is None:
            break

    return all_ids

# point_ids = get_all_point_ids(qdrant_client, collection_name="demo_collection")
# delete_by_ids(qdrant_client, point_ids, "demo_collection")
# create_collection(qdrant_client, collection_name="demo_collection", vector_size=768, distance=Distance.COSINE)
# delete_collection(qdrant_client, collection_name="demo_collection")

### Add payload & upsert

In [312]:
from uuid import uuid4

In [302]:
metadatas = [i.metadata for i in all_splits]
texts = [i.page_content for i in all_splits]

all_embeddings = embed_texts_strict_rate(gemini_client, 
                                         texts=texts,
                                         model_id = "models/text-embedding-004",
                                         task_type = "retrieval_document",
                                         batch_size = DEFAULT_BATCH_SIZE)

0it [00:00, ?it/s]

Processed batch 1/2
Processed batch 2/2


In [336]:
data_points = []

for i in tqdm(range(len(all_splits))):
    # import info
    metadata = metadatas[i]
    text = texts[i]
    embedding = all_embeddings[i]

    # get page_number
    file_extension = metadata["source"]
    if "page_label" in metadata or "page_number" in metadata:
        if file_extension == '.pdf':
            page_number = doc_metadata["page_label"]
        elif file_extension == '.docx' or file_ext == '.doc':
            page_number = doc_metadata["page_number"]
    else:
        # have only a single page or a pure txt file
        page_number = 1
        
    point_data = {}
    point_data["id"] = str(uuid4())
    point_data["vector"] = embedding
    point_data["payload"] = {"document": text,
                             "source": metadata["source"],
                             "filename": os.path.basename(metadata["source"]),
                             "page_number": page_number,
                             "chunk_id": point_data["id"]
                            }
    data_points.append(point_data)

  0%|          | 0/76 [00:00<?, ?it/s]

In [337]:
data_points[0]

{'id': '4c9ed596-a994-4844-9397-e7609c53dc46',
 'vector': [-0.0016930694,
  0.028019778,
  -0.016092042,
  -0.037501037,
  0.040755454,
  -0.012869336,
  0.046589997,
  0.046900485,
  0.0039028577,
  0.016852548,
  -0.0005425717,
  0.024911389,
  0.08421891,
  -0.031631142,
  -0.000781615,
  -0.030501349,
  0.0013308527,
  0.02144295,
  -0.098889105,
  -0.010018094,
  -0.0040234914,
  -0.03651973,
  0.06005013,
  0.0119622005,
  -0.025535783,
  -0.016367404,
  0.025854297,
  0.024775296,
  0.03484505,
  -0.007503149,
  0.010685991,
  0.08551329,
  0.025198366,
  -0.0013542844,
  0.0017628099,
  0.059415657,
  0.019935763,
  0.0101865465,
  0.04058197,
  -0.046711758,
  -0.030506704,
  0.0065191886,
  -0.020530928,
  0.049420096,
  -0.040924888,
  0.01096435,
  0.010194904,
  0.062561005,
  -0.014644876,
  0.049710814,
  0.036879342,
  0.023982221,
  -0.036074348,
  0.01968965,
  -0.046665616,
  -0.036809582,
  -0.0028360814,
  -0.016722793,
  0.031518575,
  -0.032173887,
  -0.021758316

In [338]:
from typing import List, Union, Dict, Any
from qdrant_client import QdrantClient
from qdrant_client.http import models

def upsert_points(
    client: QdrantClient,
    collection_name: str,
    points: List[Dict[str, Union[int, List[float], Dict[str, Any]]]],
) -> None:
    """
    Upsert points into a Qdrant collection.

    Args:
        client (QdrantClient): An initialized Qdrant client.
        collection_name (str): The name of the collection to insert into.
        points (List[Dict]): A list of points where each point is a dict with:
            - id (int or str): Unique identifier for the point.
            - vector (List[float]): The embedding vector.
            - payload (Dict[str, Any]): Metadata payload (optional).
    """
    point_structs = []

    for p in points:
        point_struct = models.PointStruct(
            id=p["id"],
            vector=p["vector"],
            payload=p.get("payload", {})
        )
        point_structs.append(point_struct)

    try:
        client.upsert(
            collection_name=collection_name,
            points=point_structs
        )
        print(f"✅ Successfully upserted {len(points)} points into '{collection_name}'.")
    except Exception as e:
        print(f"❌ Failed to upsert points: {e}")

In [339]:
upsert_points(qdrant_client, collection_name="demo_collection", points=data_points)

✅ Successfully upserted 76 points into 'demo_collection'.


### Function

In [ ]:
from typing import List, Optional, Dict, Union
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    PointStruct,
    Distance,
    VectorParams,
    Filter,
    ScoredPoint,
)
from qdrant_client.http.exceptions import UnexpectedResponse
from .embedding.base import BaseEmbedder

class VectorDB:
    def __init__(
        self,
        qdrant_url: str,
        collection_name: str,
        embedder: BaseEmbedder,
        prefer_grpc: bool = False,
    ):
        self.collection_name = collection_name
        self.embedder = embedder
        self.client = QdrantClient(
            url=qdrant_url,
            prefer_grpc=prefer_grpc
        )

    def create_collection(self, distance: str = "Cosine", **kwargs):
        if distance.lower() == "cosine":
            dist = Distance.COSINE
        elif distance.lower() == "euclidean":
            dist = Distance.EUCLID
        elif distance.lower() == "dot":
            dist = Distance.DOT
        else:
            raise ValueError(f"Unsupported distance metric: {distance}")

        try:
            self.client.recreate_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=self.embedder.get_dimension(),
                    distance=dist
                ),
                **kwargs
            )
            print(f"Collection '{self.collection_name}' created.")
        except UnexpectedResponse as e:
            raise RuntimeError(f"Failed to create collection: {str(e)}")

    def delete_collection(self, collection_name: Optional[str] = None):
        collection = collection_name or self.collection_name
        self.client.delete_collection(collection_name=collection)
        print(f"Collection '{collection}' deleted.")

    def list_collections(self) -> List[str]:
        collections = self.client.get_collections().collections
        return [c.name for c in collections]

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        return self.embedder.embed(texts)

    def upsert(
        self,
        ids: List[Union[int, str]],
        texts: List[str],
        metadatas: Optional[List[Dict]] = None,
    ):
        vectors = self.embed_texts(texts)

        if metadatas is None:
            metadatas = [{} for _ in texts]

        points = [
            PointStruct(id=ids[i], vector=vectors[i], payload=metadatas[i])
            for i in range(len(ids))
        ]

        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )

    def search(
        self,
        query: str,
        top_k: int = 5,
        filter: Optional[Filter] = None
    ) -> List[ScoredPoint]:
        query_vector = self.embed_texts([query])[0]
        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_vector,
            limit=top_k,
            query_filter=filter,
        )
        return results

    def delete_by_ids(self, ids: List[Union[int, str]]):
        self.client.delete(
            collection_name=self.collection_name,
            points_selector={"points": ids}
        )


# Implement RAG

### Trial

In [419]:
hits = qdrant_client.query_points(
    collection_name="demo_collection",
    query=make_embed_text_fn(gemini_client,
                             text="IS report ควรมีความยาวของบทความไม่เกิน 10 หน้าใช่หรือไม่",
                             model_id="models/text-embedding-004",
                             task_type="retrieval_query")[0],
    limit=10,
)

In [391]:
sample = hits.points[0]
print(f"score {sample.score}")
Markdown(sample.payload['document'])

score 0.65446025


นั้นนะคะก็จะเป็นบทความที่บรรยายเกี่ยว กับการศึกษาเฉพาะเรื่องที่เราทำนะคะซึ่ง จะมีความยาวไม่เกิน 10 หน้าโดยที่ไม่รวม บทขัดย่อนะคะสำหรับกระบวนการทำชต Paper  นะคะหลายคนสงสัยว่าจะมีกระบวนการสเต็ปการ ทำงานของมันอย่างไรบ้างนะคะเดี๋ยวเราจะมี อ่อตัว QR  Code ขึ้นให้บนหน้าจอนะคะเรา จะสามารถเข้าไปเห็นกระบวนการการทำชอตเปอร ได้นะคะจะมีตารางการส่งงานต่างๆให้เรา เห็นนะคะแล้วก็สามารถที่จะอ่าทำงานนะคะทำ ตามสเต็ปที่เค้าวางไว้ให้ได้เลยนะคะซึ่ง ในการเขียน Short  Paper นะคะจะใช้เตของ คณะที่จัดไว้ให้เท่านั้นนะคะก็สามารถ ดาวน์โหลดได้กันที่เว็บไซต์ของคณะและวิธี การส่งชต Paper นะคะเราก็จะมีเอ่อ เว็บไซต์นะคะในการส่งชต Paper นะคะซึ่งจะ มีการส่งเา้าเรียกว่า is  Report นะคะ หรือว่า independent  study  Report นั่น เองนะคะส่งเข้าไปลิงก์ตามที่ขึ้นให้ด้าน ล่างนี้นะคะสำหรับนักศึกษาที่เข้ามาตั้ง แต่เอ่อเดือนมกราคมปี  2565 เป็นต้นไปนะคะข้อควรระวังอย่างนึง การทำชอตเปอร์หรือการศึกษาโครงการเฉพาะ เรื่องของนักศึกษาที่เข้ามาตั้งแต่มกราคม  2565 ย้ำนะคะจะต้องผ่านการอบรมจริยธรรม การวิจัยทั้งทฤษฎีและปฏิบัติก่อนการทำงาน

In [424]:
retrieved_result = [dict(i) for i in hits.points]
retrieved_result

[{'id': '3731665a-6d04-45af-b19e-36bce2c23b65',
  'version': 3,
  'score': 0.65446025,
  'payload': {'document': 'นั้นนะคะก็จะเป็นบทความที่บรรยายเกี่ยว กับการศึกษาเฉพาะเรื่องที่เราทำนะคะซึ่ง จะมีความยาวไม่เกิน 10 หน้าโดยที่ไม่รวม บทขัดย่อนะคะสำหรับกระบวนการทำชต Paper  นะคะหลายคนสงสัยว่าจะมีกระบวนการสเต็ปการ ทำงานของมันอย่างไรบ้างนะคะเดี๋ยวเราจะมี อ่อตัว QR  Code ขึ้นให้บนหน้าจอนะคะเรา จะสามารถเข้าไปเห็นกระบวนการการทำชอตเปอร ได้นะคะจะมีตารางการส่งงานต่างๆให้เรา เห็นนะคะแล้วก็สามารถที่จะอ่าทำงานนะคะทำ ตามสเต็ปที่เค้าวางไว้ให้ได้เลยนะคะซึ่ง ในการเขียน Short  Paper นะคะจะใช้เตของ คณะที่จัดไว้ให้เท่านั้นนะคะก็สามารถ ดาวน์โหลดได้กันที่เว็บไซต์ของคณะและวิธี การส่งชต Paper นะคะเราก็จะมีเอ่อ เว็บไซต์นะคะในการส่งชต Paper นะคะซึ่งจะ มีการส่งเา้าเรียกว่า is  Report นะคะ หรือว่า independent  study  Report นั่น เองนะคะส่งเข้าไปลิงก์ตามที่ขึ้นให้ด้าน ล่างนี้นะคะสำหรับนักศึกษาที่เข้ามาตั้ง แต่เอ่อเดือนมกราคมปี  2565 เป็นต้นไปนะคะข้อควรระวังอย่างนึง การทำชอตเปอร์หรือการศึกษาโครงการเฉพาะ เรื่องของนักศึก

In [433]:
context = "\n".join(r.payload["document"] for r in hits.points)
context

'นั้นนะคะก็จะเป็นบทความที่บรรยายเกี่ยว กับการศึกษาเฉพาะเรื่องที่เราทำนะคะซึ่ง จะมีความยาวไม่เกิน 10 หน้าโดยที่ไม่รวม บทขัดย่อนะคะสำหรับกระบวนการทำชต Paper  นะคะหลายคนสงสัยว่าจะมีกระบวนการสเต็ปการ ทำงานของมันอย่างไรบ้างนะคะเดี๋ยวเราจะมี อ่อตัว QR  Code ขึ้นให้บนหน้าจอนะคะเรา จะสามารถเข้าไปเห็นกระบวนการการทำชอตเปอร ได้นะคะจะมีตารางการส่งงานต่างๆให้เรา เห็นนะคะแล้วก็สามารถที่จะอ่าทำงานนะคะทำ ตามสเต็ปที่เค้าวางไว้ให้ได้เลยนะคะซึ่ง ในการเขียน Short  Paper นะคะจะใช้เตของ คณะที่จัดไว้ให้เท่านั้นนะคะก็สามารถ ดาวน์โหลดได้กันที่เว็บไซต์ของคณะและวิธี การส่งชต Paper นะคะเราก็จะมีเอ่อ เว็บไซต์นะคะในการส่งชต Paper นะคะซึ่งจะ มีการส่งเา้าเรียกว่า is  Report นะคะ หรือว่า independent  study  Report นั่น เองนะคะส่งเข้าไปลิงก์ตามที่ขึ้นให้ด้าน ล่างนี้นะคะสำหรับนักศึกษาที่เข้ามาตั้ง แต่เอ่อเดือนมกราคมปี  2565 เป็นต้นไปนะคะข้อควรระวังอย่างนึง การทำชอตเปอร์หรือการศึกษาโครงการเฉพาะ เรื่องของนักศึกษาที่เข้ามาตั้งแต่มกราคม  2565 ย้ำนะคะจะต้องผ่านการอบรมจริยธรรม การวิจัยทั้งทฤษฎีและปฏิบัติก่อนการทำงาน\nAA-U-F09

In [431]:
import json

def list_of_json_to_string(json_list: list) -> str:
    """
    Converts a list of dictionaries (JSON objects) into a single JSON string.

    Args:
        json_list (list): A list of dictionaries.

    Returns:
        str: A JSON-formatted string.
    """
    return json.dumps(json_list, ensure_ascii=False, indent=4)

list_of_json_to_string(retrieved_result)

'[\n    {\n        "id": "3731665a-6d04-45af-b19e-36bce2c23b65",\n        "version": 3,\n        "score": 0.65446025,\n        "payload": {\n            "document": "นั้นนะคะก็จะเป็นบทความที่บรรยายเกี่ยว กับการศึกษาเฉพาะเรื่องที่เราทำนะคะซึ่ง จะมีความยาวไม่เกิน 10 หน้าโดยที่ไม่รวม บทขัดย่อนะคะสำหรับกระบวนการทำชต Paper  นะคะหลายคนสงสัยว่าจะมีกระบวนการสเต็ปการ ทำงานของมันอย่างไรบ้างนะคะเดี๋ยวเราจะมี อ่อตัว QR  Code ขึ้นให้บนหน้าจอนะคะเรา จะสามารถเข้าไปเห็นกระบวนการการทำชอตเปอร ได้นะคะจะมีตารางการส่งงานต่างๆให้เรา เห็นนะคะแล้วก็สามารถที่จะอ่าทำงานนะคะทำ ตามสเต็ปที่เค้าวางไว้ให้ได้เลยนะคะซึ่ง ในการเขียน Short  Paper นะคะจะใช้เตของ คณะที่จัดไว้ให้เท่านั้นนะคะก็สามารถ ดาวน์โหลดได้กันที่เว็บไซต์ของคณะและวิธี การส่งชต Paper นะคะเราก็จะมีเอ่อ เว็บไซต์นะคะในการส่งชต Paper นะคะซึ่งจะ มีการส่งเา้าเรียกว่า is  Report นะคะ หรือว่า independent  study  Report นั่น เองนะคะส่งเข้าไปลิงก์ตามที่ขึ้นให้ด้าน ล่างนี้นะคะสำหรับนักศึกษาที่เข้ามาตั้ง แต่เอ่อเดือนมกราคมปี  2565 เป็นต้นไปนะคะข้อควรระวังอย่างนึง ก

### Implement

In [464]:
import base64
import os
from google import genai
from google.genai import types
from google.genai.client import Client as GeminiClient

def retrieve_context(query: str,
                     collection_name: str = "demo_collection",
                     top_k: int = 5):
    hits = qdrant_client.query_points(
        collection_name="demo_collection",
        query=make_embed_text_fn(gemini_client,
                                 text=query,
                                 model_id="models/text-embedding-004",
                                 task_type="retrieval_query")[0],
        limit=top_k,
    )
    # context = "\n".join(r.payload["document"] for r in hits.points)
    retrieved_result = [dict(i) for i in hits.points]
    return retrieved_result
    

def generate(client,
             query: str,
             model_name: str = "gemini-2.5-flash-preview-04-17",
            ):
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=query),
            ],
        ),
    ]
    
    # Call the model
    response = client.models.generate_content(
        model=model_name,
        contents=contents,
    )
    
    return response.text


def rag(client: GeminiClient, 
        query: str,
        collection_name: str = "demo_collection",
        top_k: int = 3
    ) -> str:
    # Retrieve relevant context
    results = retrieve_context(query, collection_name, top_k)

    # rag prompt
    base_prompt = f"""based on the provided context, answer the following question accurately and completely in Thai language with this set of instructions.

    Instructions:
    - Provide a comprehensive answer based only on the information in the context.
    - If the context doesn't contain enough information, clearly state what information is missing.
    - Be specific and detailed in your response.
    - At the end of your answer, list all sources you referenced with their details.
    - Answer in THAI language.
    
    Context:
    {results}

    Question: 
    {query}

    Answer:
    """
    
    # Generate the answer using Gemini
    answer = generate(client=client, query=base_prompt)

    return answer

In [465]:
from google import genai
gemini_client = genai.Client()

response = rag(gemini_client, 
               query="IS report ควรมีความยาวของบทความไม่เกิน 10 หน้าใช่หรือไม่",
               top_k=3,
              )

In [468]:
print(response)

จากข้อมูลที่ให้มา ใช่ IS report หรือ Short Paper ที่บรรยายเกี่ยวกับการศึกษาเฉพาะเรื่อง ควรมีความยาวของบทความไม่เกิน 10 หน้า โดยไม่รวมบทคัดย่อ

**แหล่งข้อมูลที่อ้างอิง:**

*   Context ID: 3731665a-6d04-45af-b19e-36bce2c23b65 จากไฟล์: sit_kmutt_แนะนำการทำ_Short_Paper.txt
*   Context ID: b57a4437-b22d-48f8-a6ed-3b11e1eb2087 จากไฟล์: sit_kmutt_แนะนำการทำ_Short_Paper.txt
